# Description

Python code to scrape player data from baseball-reference.com and rate players using SherCo PLUS ratings. Ratings are effective for any season since and including 1950.

# Imports

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from lxml import html
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, parse_qs

from openpyxl import load_workbook

from bisect import bisect

# Scrape season data

In [4]:
year = '1986'

In [350]:
url_bat = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-standard-batting.shtml'
url_pit = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-standard-pitching.shtml'
url_fld = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-standard-fielding.shtml'
url_cat = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-specialpos_c-fielding.shtml'
url_app = 'https://www.baseball-reference.com/leagues/MLB/' + year + '-appearances-fielding.shtml'

In [351]:
url_cat

'https://www.baseball-reference.com/leagues/MLB/2017-specialpos_c-fielding.shtml'

In [352]:
session_requests = requests.session()

result = session_requests.get(url_bat, headers = dict(referer = url_bat))
result.status_code

200

In [353]:
# https://github.com/BenKite/baseball_data/blob/master/baseballReferenceScrape.py
def findTables(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    divs = soup.find_all('div', id = "content")
    divs = divs[0].find_all("div", id=re.compile("^all"))
    ids = []
    for div in divs:
        searchme = str(div.find_all("table"))
        x = searchme[searchme.find("id=") + 3: searchme.find(">")]
        x = x.replace("\"", "")
        if len(x) > 0:
            ids.append(x)
    return(ids)

In [354]:
findTables(url_bat)

['teams_standard_batting', 'players_standard_batting']

In [355]:
soup = BeautifulSoup(result.content, 'lxml')

In [1]:
# print(soup.prettify())

In [357]:
def pullTable(url, tableID):
    res = requests.get(url)
    ## Work around comments
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    tables = soup.find_all('table', id = tableID)
    data_rows = tables[0].find_all('tr')
    data_header = tables[0].find_all('thead')
    data_header = data_header[0].find_all("tr")
    data_header = data_header[0].find_all("th")
    game_data = [[td.getText() for td in data_rows[i].find_all(['th','td'])]
        for i in range(len(data_rows))
        ]
    data = pd.DataFrame(game_data)
    header = []
    for i in range(len(data.columns)):
        header.append(data_header[i].getText())
    data.columns = header
    data = data.loc[data[header[0]] != header[0]]
    data = data.reset_index(drop = True)
    return(data)

In [358]:
bat = pullTable(url_bat, 'players_standard_batting')

In [359]:
findTables(url_pit)

['teams_standard_pitching', 'players_standard_pitching']

In [360]:
pit = pullTable(url_pit, 'players_standard_pitching')

In [361]:
findTables(url_fld)

['teams_standard_fielding', 'players_players_standard_fielding_fielding']

In [362]:
fld = pullTable(url_fld, 'players_players_standard_fielding_fielding')

In [363]:
findTables(url_cat)

['teams_standard_fielding',
 'teams_advanced_fielding_c',
 'teams_advanced_fielding_c_baserunning',
 'players_players_standard_fielding_fielding',
 'players_players_advanced_fielding_c_fielding',
 'players_players_advanced_fielding_c_baserunning_fielding']

In [364]:
cat = pullTable(url_cat, 'players_players_standard_fielding_fielding')

In [365]:
findTables(url_app)

['teams_appearances', 'players_players_appearances_fielding']

In [366]:
app = pullTable(url_app, 'players_players_appearances_fielding')

In [367]:
bat.tail()

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary
1449,1450,Jordan Zimmermann,31,DET,AL,3,6,6,0,1,0,0,0,0,0,0,0,1,.167,.167,.167,.333,-10,1,0,0,0,0,0,1
1450,1451,Ben Zobrist#,36,CHC,NL,128,496,435,58,101,20,3,12,50,2,2,54,71,.232,.318,.375,.693,79,163,13,2,2,3,2,479H/36
1451,1452,Mike Zunino,26,SEA,AL,124,435,387,52,97,25,0,25,64,1,0,39,160,.251,.331,.509,.840,125,197,8,8,0,1,0,*2/HD
1452,1453,Tony Zych,26,SEA,AL,4,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,0,0,0,0,0,0,1
1453,,LgAvg per 600 PA,,,,200,600,536,73,136,27,3,20,70,8,3,51,130,.254,.324,.425,.749,,228,12,6,3,4,3,


In [368]:
bat = bat[bat["Name"] != "LgAvg per 600 PA"]

In [369]:
def how_bats(names):
    bats = ""
    for name in names:
        if name.rfind("#") > -1:
            bats = "S"
        elif name.rfind("*") > -1:
            bats = "L"
        else:
            bats = "R"
    return bats

In [370]:
bat["Bats"] = bat["Name"].apply(how_bats)
bat["Bats"].value_counts()

R    917
L    417
S    119
Name: Bats, dtype: int64

In [371]:
bat["Name"] = [re.sub("[*#]", "", name) for name in bat["Name"]]
bat.tail()

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Bats
1448,1449,Ryan Zimmerman,32,WSN,NL,144,576,524,90,159,33,0,36,108,1,0,44,126,.303,.358,.573,.930,134,300,16,3,0,5,1,*3/HD,R
1449,1450,Jordan Zimmermann,31,DET,AL,3,6,6,0,1,0,0,0,0,0,0,0,1,.167,.167,.167,.333,-10,1,0,0,0,0,0,1,R
1450,1451,Ben Zobrist,36,CHC,NL,128,496,435,58,101,20,3,12,50,2,2,54,71,.232,.318,.375,.693,79,163,13,2,2,3,2,479H/36,S
1451,1452,Mike Zunino,26,SEA,AL,124,435,387,52,97,25,0,25,64,1,0,39,160,.251,.331,.509,.840,125,197,8,8,0,1,0,*2/HD,R
1452,1453,Tony Zych,26,SEA,AL,4,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,0,0,0,0,0,0,1,R


In [372]:
bat.drop(columns=["Rk"], inplace=True)

In [373]:
bat.tail()

,Name,Age,Tm,Lg,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Bats
1448,Ryan Zimmerman,32,WSN,NL,144,576,524,90,159,33,0,36,108,1,0,44,126,.303,.358,.573,.930,134,300,16,3,0,5,1,*3/HD,R
1449,Jordan Zimmermann,31,DET,AL,3,6,6,0,1,0,0,0,0,0,0,0,1,.167,.167,.167,.333,-10,1,0,0,0,0,0,1,R
1450,Ben Zobrist,36,CHC,NL,128,496,435,58,101,20,3,12,50,2,2,54,71,.232,.318,.375,.693,79,163,13,2,2,3,2,479H/36,S
1451,Mike Zunino,26,SEA,AL,124,435,387,52,97,25,0,25,64,1,0,39,160,.251,.331,.509,.840,125,197,8,8,0,1,0,*2/HD,R
1452,Tony Zych,26,SEA,AL,4,0,0,0,0,0,0,0,0,0,0,0,0,,,,,,0,0,0,0,0,0,1,R


In [374]:
pit.tail()

,Rk,Name,Age,Tm,Lg,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W
919,920,Rob Zastryzny*,25,CHC,NL,0,0,,8.31,4,0,0,0,0,0,13.0,19,13,12,2,7,0,11,1,0,0,62,54,5.31,2.000,13.2,1.4,4.8,7.6,1.57
920,921,Brad Ziegler,37,MIA,NL,1,4,.200,4.79,53,0,21,0,0,10,47.0,57,29,25,1,16,6,26,6,0,1,211,83,3.73,1.553,10.9,0.2,3.1,5.0,1.63
921,922,Jordan Zimmermann,31,DET,AL,8,13,.381,6.08,29,29,0,0,0,0,160.0,204,111,108,29,44,2,103,7,0,3,713,74,5.18,1.550,11.5,1.6,2.5,5.8,2.34
922,923,Tony Zych,26,SEA,AL,6,3,.667,2.66,45,0,7,0,0,1,40.2,30,12,12,2,21,3,35,5,1,1,173,158,3.99,1.254,6.6,0.4,4.6,7.7,1.67
923,,LgAvg per 180 IP,,,,10,10,.494,4.37,88,20,21,0,0,5,180,176,94,87,25,66,4,167,7,1,8,771,101,4.36,1.343,8.8,1.3,3.3,8.3,2.54


In [375]:
pit = pit[pit["Name"] != "LgAvg per 180 IP"]

In [376]:
def how_throws(names):
    throws = ""
    for name in names:
        if name.rfind("*") > -1:
            throws = "L"
        else:
            throws = "R"
    return throws

In [377]:
pit["Throws"] = pit["Name"].apply(how_throws)
pit["Throws"].value_counts()

R    686
L    237
Name: Throws, dtype: int64

In [378]:
pit["Name"] = [re.sub("[*#]", "", name) for name in pit["Name"]]
pit.tail()

,Rk,Name,Age,Tm,Lg,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws
918,919,Chris Young,38,KCR,AL,0,0,,7.50,14,2,5,0,0,0,30.0,47,27,25,7,14,2,22,1,0,1,148,60,6.22,2.033,14.1,2.1,4.2,6.6,1.57,R
919,920,Rob Zastryzny,25,CHC,NL,0,0,,8.31,4,0,0,0,0,0,13.0,19,13,12,2,7,0,11,1,0,0,62,54,5.31,2.000,13.2,1.4,4.8,7.6,1.57,L
920,921,Brad Ziegler,37,MIA,NL,1,4,.200,4.79,53,0,21,0,0,10,47.0,57,29,25,1,16,6,26,6,0,1,211,83,3.73,1.553,10.9,0.2,3.1,5.0,1.63,R
921,922,Jordan Zimmermann,31,DET,AL,8,13,.381,6.08,29,29,0,0,0,0,160.0,204,111,108,29,44,2,103,7,0,3,713,74,5.18,1.550,11.5,1.6,2.5,5.8,2.34,R
922,923,Tony Zych,26,SEA,AL,6,3,.667,2.66,45,0,7,0,0,1,40.2,30,12,12,2,21,3,35,5,1,1,173,158,3.99,1.254,6.6,0.4,4.6,7.7,1.67,R


In [379]:
pit.drop(columns=["Rk"], inplace=True)

In [380]:
players = pd.merge(bat, pit, how="outer", on=["Name", "Tm", "Age"], suffixes=('_bat', '_pit'))

In [381]:
players.shape

(1631, 62)

In [382]:
fld.tail()

,Rk,Name,Age,Tm,Lg,G,GS,CG,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos Summary
1350,1351,Jordan Zimmermann,31,DET,AL,29,29,0,160.0,14,7,6,1,0,.929,,,-3,-4,0,0.73,0.45,P
1351,1352,Ben Zobrist,36,CHC,NL,152,105,49,915.2,340,188,149,3,39,.991,2,3,4,5,2,3.31,2.22,2B-OF-SS-1B
1352,1353,Mike Zunino,26,SEA,AL,120,112,105,994.0,957,894,56,7,5,.993,-6,-7,4,5,-4,8.60,7.92,C
1353,1354,Tony Zych,26,SEA,AL,45,0,0,40.2,4,1,3,0,0,1.000,,,0,0,0,0.89,0.09,P
1354,,LgAvg,,,,216,149,114,1324,613,445,158,10,42,.984,0,0,,,,4.10,2.80,


In [383]:
fld = fld[fld["Name"] != "LgAvg"]

In [384]:
fld.drop(columns=["Rk"], inplace=True)

In [385]:
players = pd.merge(players, fld, how="left", on=["Name", "Age"], suffixes=('', '_fld'))

In [386]:
cat.tail()

,Rk,Name,Age,Tm,Lg,G,GS,CG,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rctch,Rdrs,Rdrs/yr,Rgood,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9,RF/G,PB,WP,SB,CS,CS%
121,122,Chad Wallach,25,CIN,NL,3,2,2,21.0,22,20,1,1,0,.955,0,0,0,1,57,0,0,1,0,0,0,0,9.00,7.00,0,0,1,0,0%
122,123,Matt Wieters,31,WSN,NL,118,113,102,1003.2,1074,1023,43,8,5,.993,1,1,1,-5,-6,-2,0,-2,0,-4,0,3,9.56,9.03,5,28,57,19,25%
123,124,Tony Wolters,25,COL,NL,77,66,61,586.1,573,528,43,2,5,.997,3,6,3,0,0,1,0,1,0,-2,4,-3,8.76,7.42,7,28,22,14,39%
124,125,Mike Zunino,26,SEA,AL,120,112,105,994.0,957,894,56,7,5,.993,-6,-7,-6,4,5,-4,0,0,0,3,1,4,8.60,7.92,10,52,54,17,24%
125,,LgAvg,,,,20,18,15,158,158,148,9,1,1,.993,0,0,0,0,0,0,0,0,0,0,0,0,8.92,7.78,1,7,9,3,27%


In [387]:
cat.drop_duplicates(subset=["Name"], keep='first', inplace=True)

In [388]:
cat = cat[cat["Name"] != "LgAvg"]

In [389]:
cat.drop(columns=["Rk"], inplace=True)

In [390]:
players = pd.merge(players, cat, how='left', on=["Name", "Age"], suffixes=('', '_cat'))

In [391]:
app.tail()

,Rk,Name,Age,Tm,Yrs,G,GS,Batting,Defense,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR
1353,1354,Ryan Zimmerman,32,WSN,13,144,132,144,143,0,0,143,0,0,0,0,0,0,0,1,4,0
1354,1355,Jordan Zimmermann,31,DET,9,29,29,3,29,29,0,0,0,0,0,0,0,0,0,0,0,0
1355,1356,Ben Zobrist,36,CHC,12,128,105,128,118,0,0,5,81,0,5,36,0,32,61,0,20,0
1356,1357,Mike Zunino,26,SEA,5,124,112,124,120,0,120,0,0,0,0,0,0,0,0,2,4,1
1357,1358,Tony Zych,26,SEA,3,45,0,4,45,45,0,0,0,0,0,0,0,0,0,0,0,0


In [392]:
players = pd.merge(players, app, how='left', on=["Name", "Age"], suffixes=('', '_app'))

In [393]:
players.to_csv("../data/player stats" + " - " + year + ".csv", index=False)

# Rate Players

In [394]:
players = pd.read_csv("../data/player stats - " + year + ".csv")
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR
0,Fernando Abad,31,BOS,AL,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,L,AL,2.0,1.0,0.667,3.3,48.0,0.0,15.0,0.0,0.0,1.0,43.2,40.0,18.0,16.0,4.0,14.0,1.0,37.0,1.0,1.0,0.0,182.0,139.0,3.68,1.237,8.2,0.8,2.9,7.6,2.64,L,BOS,AL,48.0,0.0,0.0,43.2,6.0,0.0,6.0,0.0,1.0,1.000,NaN,NaN,1.0,5.0,0.0,1.24,0.13,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BOS,8,48,0,4,48,48,0,0,0,0,0,0,0,0,0,0,0,0
1,José Abreu,30,CHW,AL,156.0,675.0,621.0,95.0,189.0,43.0,6.0,33.0,102.0,3.0,0.0,35.0,119.0,0.304,0.354,0.552,0.906,141.0,343.0,21.0,15.0,0.0,4.0,6.0,*3D,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHW,AL,139.0,138.0,130.0,1197.0,1221.0,1135.0,78.0,8.0,130.0,0.993,3.0,3.0,2.0,2.0,-1.0,9.12,8.73,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,CHW,4,156,156,156,139,0,0,139,0,0,0,0,0,0,0,18,0,0
2,Cristhian Adames,25,COL,NL,12.0,14.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.000,0.071,0.000,0.071,-80.0,0.0,0.0,0.0,0.0,0.0,0.0,H/346,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COL,NL,3.0,1.0,0.0,9.0,4.0,3.0,1.0,0.0,0.0,1.000,-1.0,-93.0,-1.0,-133.0,0.0,4.00,1.33,SS-2B-1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,COL,4,12,1,12,2,0,0,1,1,0,1,0,0,0,0,0,10,0
3,Austin Adams,26,WSN,NL,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,3.6,6.0,0.0,3.0,0.0,0.0,0.0,5.0,4.0,4.0,2.0,0.0,8.0,0.0,10.0,1.0,0.0,1.0,29.0,133.0,4.56,2.400,7.2,0.0,14.4,18.0,1.25,R,WSN,NL,6.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.000,NaN,NaN,0.0,0.0,0.0,1.80,0.17,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,WSN,1st,6,0,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0
4,Lane Adams,27,ATL,NL,85.0,122.0,109.0,19.0,30.0,4.0,1.0,5.0,20.0,10.0,0.0,10.0,37.0,0.275,0.339,0.468,0.807,109.0,51.0,3.0,1.0,1.0,1.0,0.0,H78/9,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NL,41.0,13.0,9.0,172.0,51.0,48.0,1.0,2.0,0.0,0.961,1.0,5.0,NaN,NaN,NaN,2.56,1.20,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,ATL,2,85,13,85,41,0,0,0,0,0,0,27,11,7,41,0,51,9


In [395]:
pd.set_option('max_seq_items', 200)
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B',
       ...
       '2B_app', '3B_app', 'SS', 'LF', 'CF', 'RF', 'OF', 'DH', 'PH', 'PR'],
      dtype='object', length=133)

In [396]:
list(players.columns.values)

['Name',
 'Age',
 'Tm',
 'Lg_bat',
 'G_bat',
 'PA',
 'AB',
 'R_bat',
 'H_bat',
 '2B',
 '3B',
 'HR_bat',
 'RBI',
 'SB',
 'CS',
 'BB_bat',
 'SO_bat',
 'BA',
 'OBP',
 'SLG',
 'OPS',
 'OPS+',
 'TB',
 'GDP',
 'HBP_bat',
 'SH',
 'SF',
 'IBB_bat',
 'Pos\xa0Summary',
 'Bats',
 'Lg_pit',
 'W',
 'L',
 'W-L%',
 'ERA',
 'G_pit',
 'GS',
 'GF',
 'CG',
 'SHO',
 'SV',
 'IP',
 'H_pit',
 'R_pit',
 'ER',
 'HR_pit',
 'BB_pit',
 'IBB_pit',
 'SO_pit',
 'HBP_pit',
 'BK',
 'WP',
 'BF',
 'ERA+',
 'FIP',
 'WHIP',
 'H9',
 'HR9',
 'BB9',
 'SO9',
 'SO/W',
 'Throws',
 'Tm_fld',
 'Lg',
 'G',
 'GS_fld',
 'CG_fld',
 'Inn',
 'Ch',
 'PO',
 'A',
 'E',
 'DP',
 'Fld%',
 'Rtot',
 'Rtot/yr',
 'Rdrs',
 'Rdrs/yr',
 'Rgood',
 'RF/9',
 'RF/G',
 'Pos\xa0Summary_fld',
 'Tm_cat',
 'Lg_cat',
 'G_cat',
 'GS_cat',
 'CG_cat',
 'Inn_cat',
 'Ch_cat',
 'PO_cat',
 'A_cat',
 'E_cat',
 'DP_cat',
 'Fld%_cat',
 'Rtot_cat',
 'Rtot/yr_cat',
 'Rctch',
 'Rdrs_cat',
 'Rdrs/yr_cat',
 'Rgood_cat',
 'Rair',
 'Rrange',
 'Rthrow',
 'RszC',
 'RsbC',
 'Re

In [397]:
players.rename(columns={
    "Pos\xa0Summary": "Pos_Summary",
    "Pos\xa0Summary_fld": "Pos_Summary_fld"
}, inplace=True)

In [398]:
players["Primary_Pos_fld"] = players['Pos_Summary_fld'].str.split("-").str[0]
players["Primary_Pos_fld"].value_counts()

P     903
OF    284
C     118
2B     88
3B     83
SS     77
1B     74
Name: Primary_Pos_fld, dtype: int64

## Batter Ratings

In [2]:
# pd.set_option('display.max_columns', 200)
# players.columns

### Clutch Rating

In [400]:
players["rbi_per_g"] = players["RBI"] / players["G_bat"]
players["rbi_per_g"].value_counts()

0.000000    677
0.333333     17
0.500000     15
0.200000     12
0.166667      8
           ... 
0.601852      1
0.321429      1
0.671642      1
0.309278      1
0.539568      1
Name: rbi_per_g, Length: 536, dtype: int64

In [401]:
players["clutch"] = (round(players["rbi_per_g"], 3) >= .6).astype(int)
players["clutch"] = players["clutch"].map({0: "", 1: "#"}).astype(str)
players["clutch"].value_counts()

     1543
#      88
Name: clutch, dtype: int64

In [402]:
players[players["clutch"] == "#"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch
1,José Abreu,30,CHW,AL,156.0,675.0,621.0,95.0,189.0,43.0,6.0,33.0,102.0,3.0,0.0,35.0,119.0,0.304,0.354,0.552,0.906,141.0,343.0,21.0,15.0,0.0,4.0,6.0,*3D,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHW,AL,139.0,138.0,130.0,1197.0,1221.0,1135.0,78.0,8.0,130.0,0.993,3.0,3.0,2.0,2.0,-1.0,9.12,8.73,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,CHW,4,156,156,156,139,0,0,139,0,0,0,0,0,0,0,18,0,0,1B,0.653846,#
13,Andrew Albers,31,SEA,AL,1.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000,1.000,1.000,2.000,453.0,1.0,0.0,0.0,2.0,0.0,0.0,/1,R,AL,5.0,1.0,0.833,3.51,9.0,6.0,2.0,0.0,0.0,1.0,41.0,43.0,22.0,16.0,6.0,10.0,0.0,37.0,2.0,0.0,1.0,178.0,119.0,4.13,1.293,9.4,1.3,2.2,8.1,3.7,L,SEA,AL,9.0,6.0,0.0,41.0,6.0,1.0,5.0,0.0,0.0,1.000,NaN,NaN,1.0,5.0,0.0,1.32,0.67,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,SEA,4,9,6,1,9,9,0,0,0,0,0,0,0,0,0,0,0,0,P,1.000000,#
31,Aaron Altherr,26,PHI,NL,107.0,412.0,372.0,58.0,101.0,24.0,5.0,19.0,65.0,5.0,4.0,32.0,104.0,0.272,0.340,0.516,0.856,122.0,192.0,12.0,7.0,0.0,1.0,2.0,798/H,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHI,NL,106.0,92.0,73.0,837.1,221.0,211.0,7.0,3.0,4.0,0.986,-4.0,-5.0,NaN,NaN,NaN,2.34,2.06,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,PHI,4,107,92,107,106,0,0,0,0,0,0,52,19,50,106,0,7,0,OF,0.607477,#
45,Miguel Andujar,22,NYY,AL,5.0,8.0,7.0,0.0,4.0,2.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.571,0.625,0.857,1.482,286.0,6.0,0.0,0.0,0.0,0.0,0.0,/5DH,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NYY,AL,3.0,0.0,0.0,9.0,2.0,1.0,1.0,0.0,0.0,1.000,0.0,27.0,0.0,0.0,0.0,2.00,0.67,3B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,NYY,1st,5,1,5,3,0,0,0,0,3,0,0,0,0,0,2,1,0,3B,0.800000,#
49,Nori Aoki,35,TOR,AL,12.0,34.0,32.0,4.0,9.0,1.0,0.0,3.0,8.0,0.0,0.0,1.0,5.0,0.281,0.294,0.594,0.888,128.0,19.0,0.0,0.0,0.0,1.0,0.0,/97H8,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TOT,MLB,105.0,87.0,56.0,766.0,141.0,133.0,7.0,1.0,1.0,0.993,-11.0,-18.0,NaN,0.0,NaN,1.64,1.33,OF-P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,3TM,6,110,87,109,105,1,0,0,0,0,0,64,2,47,104,1,10,1,OF,0.666667,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

### Letter Rating

In [403]:
players["BA"].value_counts()

0.000    201
0.200     23
0.250     22
0.167     18
0.231     17
        ... 
0.134      1
0.199      1
0.340      1
0.106      1
0.370      1
Name: BA, Length: 237, dtype: int64

In [404]:
players["BA"].isnull().sum()

530

In [405]:
players["BA"].replace(np.nan, 0.000, inplace=True)

In [406]:
break_points = [
    0.029,
    0.057,
    0.084,
    0.112,
    0.140,
    0.168,
    0.196,
    0.223,
    0.251,
    0.279,
    0.307,
    0.335,
    0.362,
    0.390
]

letters = [
    "G",
    "G+",
    "F",
    "E",
    "E+",
    "D",
    "D+",
    "C",
    "C+",
    "B",
    "B+",
    "A",
    "A+",
    "AA",
    "AAA"
]

def batter_letter(bat_avg, breakpoints=break_points, letter_grades=letters):
    i = bisect(breakpoints, bat_avg)
    return letter_grades[i]

In [407]:
players["bat_letter"] = [batter_letter(avg) for avg in players["BA"]]
players["bat_letter"].value_counts()

G      733
B      185
C+     168
C      115
B+      97
D       81
D+      58
A       48
E       37
AAA     29
E+      28
F       25
G+      16
A+       7
AA       4
Name: bat_letter, dtype: int64

In [408]:
players[players["bat_letter"] == "AAA"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter
13,Andrew Albers,31,SEA,AL,1.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000,1.000,1.000,2.000,453.0,1.0,0.0,0.0,2.0,0.0,0.0,/1,R,AL,5.0,1.0,0.833,3.51,9.0,6.0,2.0,0.0,0.0,1.0,41.0,43.0,22.0,16.0,6.0,10.0,0.0,37.0,2.0,0.0,1.0,178.0,119.0,4.13,1.293,9.4,1.3,2.2,8.1,3.70,L,SEA,AL,9.0,6.0,0.0,41.0,6.0,1.0,5.0,0.0,0.0,1.000,NaN,NaN,1.0,5.0,0.0,1.32,0.67,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,SEA,4,9,6,1,9,9,0,0,0,0,0,0,0,0,0,0,0,0,P,1.000000,#,AAA
45,Miguel Andujar,22,NYY,AL,5.0,8.0,7.0,0.0,4.0,2.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.571,0.625,0.857,1.482,286.0,6.0,0.0,0.0,0.0,0.0,0.0,/5DH,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NYY,AL,3.0,0.0,0.0,9.0,2.0,1.0,1.0,0.0,0.0,1.000,0.0,27.0,0.0,0.0,0.0,2.00,0.67,3B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,NYY,1st,5,1,5,3,0,0,0,0,3,0,0,0,0,0,2,1,0,3B,0.800000,#,AAA
126,Aaron Blair,25,ATL,NL,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000,1.000,1.000,2.000,429.0,1.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,1.0,0.000,15.00,1.0,1.0,0.0,0.0,0.0,0.0,3.0,5.0,5.0,5.0,1.0,5.0,0.0,3.0,0.0,0.0,1.0,19.0,32.0,10.49,3.333,15.0,3.0,15.0,9.0,0.60,R,ATL,NL,1.0,1.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,1.000,NaN,NaN,0.0,0.0,0.0,3.00,1.00,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119,ATL,2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,,AAA
170,Jake Buchanan,27,CIN,NL,4.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.500,0.667,0.500,1.167,213.0,1.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,8.16,5.0,0.0,3.0,0.0,0.0,0.0,14.1,24.0,13.0,13.0,1.0,7.0,2.0,4.0,3.0,0.0,0.0,77.0,56.0,5.60,2.163,15.1,0.6,4.4,2.5,0.57,R,CIN,NL,5.0,0.0,0.0,14.1,4.0,0.0,3.0,1.0,0.0,0.750,NaN,NaN,0.0,0.0,0.0,1.88,0.60,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164,CIN,4,5,0,4,5,5,0,0,0,0,0,0,0,0,0,0,0,0,P,0.250000,,AAA
225,Darrell Ceciliani,27,TOR,AL,3.0,5.0,5.0,2.0,2.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.400,0.400,1.200,1.600,301.0,6.0,0.0,0.0,0.0,0.0,0.0,/H8D7,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TOR,AL,2.0,1.0,0.0,6.0,2.0,2.0,0.0,0.0,0.0,1.000,1.0,120.0,NaN,NaN,NaN,3.00,1.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,213,TOR,3,3,1,3,2,0,0,0,0,0,0,1,1,0,2,1,1,1,OF,1.000000,#,AAA
295,Chase d'Arnaud,30,BOS,AL,2.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000,1.000,1.000,2.000,430.0,1.0,0.0,0.0,0.0,0.0,0.0,/H4,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TOT,MLB,23.0,12.0,8.0,116.1,54.0,25.0,26.0,3.0,10.0,0.944,-5.0,-51.0,NaN,0.0,NaN,3.95,2.22,SS-2B-OF-3B-1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,277,3TM,6,35,11,35,21,0,0,1,4,3,10,4

In [409]:
players[players["bat_letter"] == "C+"]["BA"].min()

0.223

### HR Number

In [410]:
players["hr_rate"] = round(players["HR_bat"] / players["H_bat"] * 36, 0)
players["hr_rate"].replace(np.nan, 0, inplace=True)
players["hr_rate"] = players["hr_rate"].astype(int)
players["hr_rate"].value_counts()

0     1032
4      100
3       74
5       72
6       71
7       65
2       60
9       32
8       31
10      21
1       19
12      17
18      10
11       6
13       6
14       5
15       3
24       2
16       2
36       2
19       1
Name: hr_rate, dtype: int64

In [411]:
players["hr_num_bat"] = players["hr_rate"].map({
    0: "",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["hr_num_bat"].value_counts()

      1032
14     100
13      74
15      72
16      71
21      65
12      60
23      32
22      31
24      21
11      19
26      17
36      10
25       6
31       6
32       5
33       3
34       2
46       2
66       2
41       1
Name: hr_num_bat, dtype: int64

### Triple Number

In [412]:
players["triple_rate"] = round(players["3B"] / players["H_bat"] * 36, 0)
players["triple_rate"].replace(np.nan, 0, inplace=True)
players["triple_rate"] = players["triple_rate"].astype(int)
players["triple_rate"].value_counts()

0     1350
1      169
2       68
3       15
4        9
5        5
12       4
9        4
7        4
6        2
29       1
Name: triple_rate, dtype: int64

In [413]:
players.loc[(players["triple_rate"] == 0), "triple_val"] = 0
players.loc[(players["triple_rate"] > 0), "triple_val"] = players["hr_rate"] + players["triple_rate"]
players["triple_val"].value_counts()

0.0     1350
6.0       42
4.0       40
5.0       35
7.0       35
8.0       26
3.0       25
2.0       20
9.0       20
10.0      14
12.0       6
1.0        5
11.0       5
13.0       2
16.0       2
21.0       1
18.0       1
17.0       1
29.0       1
Name: triple_val, dtype: int64

In [414]:
players["triple_num"] = players["triple_val"].map({
    0: "",
    1: "(11)",
    2: "(12)",
    3: "(13)",
    4: "(14)",
    5: "(15)",
    6: "(16)",
    7: "(21)",
    8: "(22)",
    9: "(23)",
    10: "(24)",
    11: "(25)",
    12: "(26)",
    13: "(31)",
    14: "(32)",
    15: "(33)",
    16: "(34)",
    17: "(35)",
    18: "(36)",
    19: "(41)",
    20: "(42)",
    21: "(43)",
    22: "(44)",
    23: "(45)",
    24: "(46)",
    25: "(51)",
    26: "(52)",
    27: "(53)",
    28: "(54)",
    29: "(55)",
    30: "(56)",
    31: "(61)",
    32: "(62)",
    33: "(63)",
    34: "(64)",
    35: "(65)",
    36: "(66)"
}).astype(str)
players["triple_num"].value_counts()

        1350
(16)      42
(14)      40
(15)      35
(21)      35
(22)      26
(13)      25
(12)      20
(23)      20
(24)      14
(26)       6
(11)       5
(25)       5
(31)       2
(34)       2
(55)       1
(36)       1
(35)       1
(43)       1
Name: triple_num, dtype: int64

### Speed Rating

In [3]:
# pd.set_option('display.max_seq_items', 200)
# players.columns

In [416]:
players["speed_score"] = round(players["SB"] / ((players["H_bat"] + players["BB_bat"] + players["HBP_bat"]) - \
                                          (players["2B"] + players["3B"] + players["HR_bat"])), 3)
players["speed_score"].replace(np.nan, 0.000, inplace=True)
players["speed_score"].value_counts()

0.000000    1193
0.100000       9
0.111111       7
0.047619       6
0.045455       6
            ... 
0.051282       1
0.005376       1
0.081301       1
0.047170       1
0.078652       1
Name: speed_score, Length: 308, dtype: int64

In [417]:
speed_breaks = [
    0.075,
    0.100,
    0.200,
    0.300
]

ratings = [
    "",
    "*",
    "**",
    "***",
    "****"
]

def speed_rate(speed, breakpoints=speed_breaks, speed_rates=ratings):
    i = bisect(breakpoints, speed)
    return speed_rates[i]

In [418]:
players["speed_rating"] = [speed_rate(rate) for rate in players["speed_score"]]
players["speed_rating"].value_counts()

        1448
**        85
*         44
***       30
****      24
Name: speed_rating, dtype: int64

In [419]:
players[players["speed_rating"] == "****"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating
4,Lane Adams,27,ATL,NL,85.0,122.0,109.0,19.0,30.0,4.0,1.0,5.0,20.0,10.0,0.0,10.0,37.0,0.275,0.339,0.468,0.807,109.0,51.0,3.0,1.0,1.0,1.0,0.0,H78/9,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NL,41.0,13.0,9.0,172.0,51.0,48.0,1.0,2.0,0.0,0.961,1.0,5.0,NaN,NaN,NaN,2.56,1.20,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,ATL,2,85,13,85,41,0,0,0,0,0,0,27,11,7,41,0,51,9,OF,0.235294,,B,6,16,1,7.0,(21),0.322581,****
45,Miguel Andujar,22,NYY,AL,5.0,8.0,7.0,0.0,4.0,2.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.571,0.625,0.857,1.482,286.0,6.0,0.0,0.0,0.0,0.0,0.0,/5DH,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NYY,AL,3.0,0.0,0.0,9.0,2.0,1.0,1.0,0.0,0.0,1.000,0.0,27.0,0.0,0.0,0.0,2.00,0.67,3B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,NYY,1st,5,1,5,3,0,0,0,0,3,0,0,0,0,0,2,1,0,3B,0.800000,#,AAA,0,,0,0.0,,0.333333,****
115,Quintin Berry,32,MIL,NL,7.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.000,0.000,0.000,0.000,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,/H78,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIL,NL,3.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,1.000,1.0,288.0,NaN,NaN,NaN,5.40,1.00,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108,MIL,5,7,0,7,3,0,0,0,0,0,0,2,1,0,3,0,1,5,OF,0.000000,,G,0,,0,0.0,,inf,****
292,Chase d'Arnaud,30,TOT,MLB,35.0,62.0,58.0,12.0,11.0,2.0,0.0,1.0,3.0,5.0,1.0,4.0,20.0,0.190,0.242,0.276,0.518,38.0,16.0,0.0,0.0,0.0,0.0,1.0,H6/47539,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TOT,MLB,23.0,12.0,8.0,116.1,54.0,25.0,26.0,3.0,10.0,0.944,-5.0,-51.0,NaN,0.0,NaN,3.95,2.22,SS-2B-OF-3B-1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,277,3TM,6,35,11,35,21,0,0,1,4,3,10,4,0,1,5,0,13,7,SS,0.085714,,D+,3,13,0,0.0,,0.416667,****
293,Chase d'Arnaud,30,TOT,NL,33.0,61.0,57.0,10.0,10.0,2.0,0.0,1.0,3.0,5.0,1.0,4.0,20.0,0.175,0.230,0.263,0.493,31.0,15.0,0.0,0.0,0.0,0.0,1.0,H6/74539,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TOT,MLB,23.0,12.0,8.0,116.1,54.0,25.0,26.0,3.0,10.0,0.944,-5.0,-51.0,NaN,0.0,NaN,3.95,2.22,SS-2B-OF-3B-1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,277,3TM,6,35,11,35,21,0,0,1,4,3,10,4,0,1,5,0,13,7,SS,0.090909,,D+,4,14,0,0.0,,0.454545,****
296,Chase d'Arnaud,30,SDP,NL,22.0,51.0,49.0,5.0,7.0,2.0,0.0,1.0,3.0,5.0,1.0,2.0,17.0,0.143,0.176,0.245,0.421,11.0,12.0,0.0,0.0,0.0,0.0,1.0,6/H4539,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

### Base on Balls Number

In [420]:
players["walk_rate"] = round(players["BB_bat"] / players["PA"] * 36, 0)
players["walk_rate"].replace(np.nan, 0, inplace=True)
players["walk_rate"] = players["walk_rate"].astype(int)
players["walk_rate"].value_counts()

0     850
2     210
3     189
4     160
1     100
5      67
6      18
7      12
8       6
18      5
9       5
36      4
12      2
10      2
14      1
Name: walk_rate, dtype: int64

In [421]:
players["bb_num"] = players["walk_rate"].map({
    0: "n",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["bb_num"].value_counts()

n     850
12    210
13    189
14    160
11    100
15     67
16     18
21     12
22      6
23      5
36      5
66      4
26      2
24      2
32      1
Name: bb_num, dtype: int64

### Batter K Number

In [422]:
players["k_rate"] = round(players["SO_bat"] / players["PA"] * 36, 0)
players["k_rate"].replace(np.nan, 0, inplace=True)
players["k_rate"] = players["k_rate"].astype(int)
players["k_rate"].value_counts()

0     595
8     148
7     102
9      89
6      81
10     76
5      69
11     69
12     64
36     55
18     51
14     42
13     41
4      30
15     20
24     13
16     13
22     10
19      9
21      9
3       9
17      8
26      5
20      5
27      4
25      3
30      3
29      2
1       2
31      1
23      1
28      1
2       1
Name: k_rate, dtype: int64

In [423]:
players.loc[(players["k_rate"] == 0), "k_val"] = 0
players.loc[(players["k_rate"] > 0), "k_val"] = players["walk_rate"] + players["k_rate"]
players["k_val"].value_counts()

0.0     595
11.0    113
12.0    104
10.0     96
13.0     83
14.0     75
9.0      74
8.0      69
18.0     65
36.0     59
16.0     51
7.0      41
15.0     40
6.0      31
17.0     25
20.0     17
24.0     16
22.0     14
19.0     12
5.0      10
21.0     10
27.0      7
26.0      6
4.0       5
25.0      3
28.0      2
30.0      2
29.0      1
1.0       1
32.0      1
2.0       1
31.0      1
23.0      1
Name: k_val, dtype: int64

In [424]:
players["k_num"] = players["k_val"].map({
    0: "n",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["k_num"].value_counts()

n     595
25    113
26    104
24     96
31     83
32     75
23     74
22     69
36     65
66     59
34     51
21     41
33     40
16     31
35     25
42     17
46     16
44     14
41     12
15     10
43     10
53      7
52      6
14      5
51      3
56      2
54      2
55      1
11      1
62      1
45      1
12      1
61      1
Name: k_num, dtype: int64

### Batter HBP Rating

In [425]:
players["hbp_rate"] = round(players["HBP_bat"] / players["PA"] * 36, 0)
players["hbp_rate"].replace(np.nan, 0, inplace=True)
players["hbp_rate"] = players["hbp_rate"].astype(int)
players["hbp_rate"].value_counts()

0    1446
1     162
2      15
4       4
3       2
6       1
5       1
Name: hbp_rate, dtype: int64

In [426]:
players.loc[(players["hbp_rate"] == 0), "hbp_val"] = 0
players.loc[(players["hbp_rate"] > 0), "hbp_val"] = players["k_val"] + players["hbp_rate"]

In [427]:
players["hbp_val"].value_counts()

0.0     1446
12.0      27
13.0      25
11.0      24
14.0      20
10.0      13
9.0       11
15.0      11
16.0      11
17.0      10
19.0       9
18.0       5
7.0        4
20.0       3
21.0       2
22.0       2
8.0        2
24.0       2
26.0       1
23.0       1
6.0        1
5.0        1
Name: hbp_val, dtype: int64

In [428]:
players["hbp_num"] = players["hbp_val"].map({
    0: "",
    1: "/11",
    2: "/12",
    3: "/13",
    4: "/14",
    5: "/15",
    6: "/16",
    7: "/21",
    8: "/22",
    9: "/23",
    10: "/24",
    11: "/25",
    12: "/26",
    13: "/31",
    14: "/32",
    15: "/33",
    16: "/34",
    17: "/35",
    18: "/36",
    19: "/41",
    20: "/42",
    21: "/43",
    22: "/44",
    23: "/45",
    24: "/46",
    25: "/51",
    26: "/52",
    27: "/53",
    28: "/54",
    29: "/55",
    30: "/56",
    31: "/61",
    32: "/62",
    33: "/63",
    34: "/64",
    35: "/65",
    36: "/66"
}).astype(str)
players["hbp_num"].value_counts()

       1446
/26      27
/31      25
/25      24
/32      20
/24      13
/34      11
/23      11
/33      11
/35      10
/41       9
/36       5
/21       4
/42       3
/22       2
/46       2
/43       2
/44       2
/15       1
/52       1
/45       1
/16       1
Name: hbp_num, dtype: int64

### Probable Hit Number

In [429]:
players["hit_rate"] = round(players["H_bat"] / players["PA"] * 36, 0)
players["hit_rate"].replace(np.nan, 0, inplace=True)
players["hit_rate"] = players["hit_rate"].astype(int)
players["hit_rate"].value_counts()

0     731
8     196
9     166
7     135
6      79
5      75
10     68
4      51
3      36
2      24
11     20
12     14
18     12
1      12
36      8
14      2
24      2
Name: hit_rate, dtype: int64

In [430]:
players["PH_num_bat"] = players["hit_rate"].map({
    0: "66",
    1: "66",
    2: "65",
    3: "64",
    4: "63",
    5: "62",
    6: "61",
    7: "56",
    8: "55",
    9: "54",
    10: "53",
    11: "52",
    12: "51",
    13: "46",
    14: "45",
    15: "44",
    16: "43",
    17: "42",
    18: "41",
    19: "36",
    20: "35",
    21: "34",
    22: "33",
    23: "32",
    24: "31",
    25: "26",
    26: "25",
    27: "24",
    28: "23",
    29: "22",
    30: "21",
    31: "16",
    32: "15",
    33: "14",
    34: "13",
    35: "12",
    36: "11"
}).astype(str)
players["PH_num_bat"].value_counts()

66    743
55    196
54    166
56    135
61     79
62     75
53     68
63     51
64     36
65     24
52     20
51     14
41     12
11      8
45      2
31      2
Name: PH_num_bat, dtype: int64

### Batter Rating

In [431]:
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B',
       ...
       'walk_rate', 'bb_num', 'k_rate', 'k_val', 'k_num', 'hbp_rate',
       'hbp_val', 'hbp_num', 'hit_rate', 'PH_num_bat'],
      dtype='object', length=154)

In [432]:
players["batter_rating"] = (players["clutch"] + players["bat_letter"] + \
                            players["hr_num_bat"] + players["triple_num"] + \
                            players["speed_rating"] + " (" + players["bb_num"] + \
                            "-" + players["k_num"] + players["hbp_num"] + ")")
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating
0,Fernando Abad,31,BOS,AL,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,L,AL,2.0,1.0,0.667,3.3,48.0,0.0,15.0,0.0,0.0,1.0,43.2,40.0,18.0,16.0,4.0,14.0,1.0,37.0,1.0,1.0,0.0,182.0,139.0,3.68,1.237,8.2,0.8,2.9,7.6,2.64,L,BOS,AL,48.0,0.0,0.0,43.2,6.0,0.0,6.0,0.0,1.0,1.000,NaN,NaN,1.0,5.0,0.0,1.24,0.13,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BOS,8,48,0,4,48,48,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,,G,0,,0,0.0,,0.000000,,0,n,0,0.0,n,0,0.0,,0,66,G (n-n)
1,José Abreu,30,CHW,AL,156.0,675.0,621.0,95.0,189.0,43.0,6.0,33.0,102.0,3.0,0.0,35.0,119.0,0.304,0.354,0.552,0.906,141.0,343.0,21.0,15.0,0.0,4.0,6.0,*3D,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHW,AL,139.0,138.0,130.0,1197.0,1221.0,1135.0,78.0,8.0,130.0,0.993,3.0,3.0,2.0,2.0,-1.0,9.12,8.73,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,CHW,4,156,156,156,139,0,0,139,0,0,0,0,0,0,0,18,0,0,1B,0.653846,#,B+,6,16,1,7.0,(21),0.019108,,2,12,6,8.0,22,1,9.0,/23,10,53,#B+16(21) (12-22/23)
2,Cristhian Adames,25,COL,NL,12.0,14.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.000,0.071,0.000,0.071,-80.0,0.0,0.0,0.0,0.0,0.0,0.0,H/346,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COL,NL,3.0,1.0,0.0,9.0,4.0,3.0,1.0,0.0,0.0,1.000,-1.0,-93.0,-1.0,-133.0,0.0,4.00,1.33,SS-2B-1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,COL,4,12,1,12,2,0,0,1,1,0,1,0,0,0,0,0,10,0,SS,0.000000,,G,0,,0,0.0,,0.000000,,3,13,15,18.0,36,0,0.0,,0,66,G (13-36)
3,Austin Adams,26,WSN,NL,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,3.6,6.0,0.0,3.0,0.0,0.0,0.0,5.0,4.0,4.0,2.0,0.0,8.0,0.0,10.0,1.0,0.0,1.0,29.0,133.0,4.56,2.400,7.2,0.0,14.4,18.0,1.25,R,WSN,NL,6.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.000,NaN,NaN,0.0,0.0,0.0,1.80,0.17,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,WSN,1st,6,0,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,,G,0,,0,0.0,,0.000000,,0,n,0,0.0,n,0,0.0,,0,66,G (n-n)
4,Lane Adams,27,ATL,NL,85.0,122.0,109.0,19.0,30.0,4.0,1.0,5.0,20.0,10.0,0.0,10.0,37.0,0.275,0.339,0.468,0.807,109.0,51.0,3.0,1.0,1.0,1.0,0.0,H78/9,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NL,41.0,13.0,9.0,172.0,51.0,48.0,1.0,2.0,0.0,0.961,1.0,5.0,NaN,NaN,NaN,2.56,1.20,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,ATL,2,85,13,85,41,0,0,0,0,0,0,27,11,7,41,0,51,9,OF,0.235294,,B,6,16,1,7.0,(21),0.322581,****,3,13,11,14.0,32,0,0.0,,9,54,B16(21)**** (13-32)


In [433]:
players.tail()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating
1626,Blake Wood,31,LAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,2.0,0.0,1.00,4.76,17.0,0.0,2.0,0.0,0.0,0.0,17.0,20.0,9.0,9.0,3.0,4.0,1.0,22.0,0.0,0.0,1.0,73.0,91.0,3.57,1.412,10.6,1.6,2.1,11.6,5.50,R,TOT,ZZ,72.0,0.0,0.0,74.1,16.0,9.0,7.0,0.0,0.0,1.000,NaN,NaN,NaN,NaN,NaN,1.94,0.22,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1333,2TM,6,72,0,52,72,72,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G (n-n)
1627,Hunter Wood,23,TBR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,0.0,NaN,0.00,1.0,0.0,1.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,3.16,0.000,0.0,0.0,0.0,0.0,NaN,R,TBR,AL,1.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.00,0.00,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1334,TBR,1st,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G (n-n)
1628,Daniel Wright,26,LAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,1.0,0.00,4.58,5.0,2.0,3.0,0.0,0.0,0.0,19.2,21.0,12.0,10.0,1.0,8.0,0.0,11.0,2.0,0.0,1.0,85.0,94.0,4.23,1.475,9.6,0.5,3.7,5.0,1.38,R,LAA,AL,5.0,2.0,0.0,19.2,3.0,1.0,2.0,0.0,0.0,1.000,NaN,NaN,1.0,10.0,0.0,1.37,0.60,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1339,LAA,2,5,2,0,5,5,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G (n-n)
1629,Steven Wright,32,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,1.0,3.0,0.25,8.25,5.0,5.0,0.0,0.0,0.0,0.0,24.0,40.0,24.0,22.0,9.0,5.0,1.0,13.0,2.0,0.0,1.0,114.0,56.0,7.82,1.875,15.0,3.4,1.9,4.9,2.60,R,BOS,AL,5.0,5.0,0.0,24.0,3.0,2.0,0.0,1.0,0.0,0.667,NaN,NaN,-1.0,-8.0,0.0,0.75,0.40,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1341,BOS,5,5,5,0,5,5,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G (n-n)
1630,Kirby Yates,30,LAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,0.0,NaN,18.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,32.0,27.16,2.000,18.0,18.0,0.0,9.0,NaN,R,TOT,ZZ,62.0,0.0,0.0,56.2,2.0,0.0,2.0,0.0,0.0,1.000,NaN,NaN,NaN,NaN,NaN,0.32,0.03,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1343,2TM,4,62,0,55,62,62,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,,G,0,,0,0.0,,0.0,,0,n,0,0.0,n,0,0.0,,0,66,G (n-n)


In [434]:
players

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating
0,Fernando Abad,31,BOS,AL,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,L,AL,2.0,1.0,0.667,3.30,48.0,0.0,15.0,0.0,0.0,1.0,43.2,40.0,18.0,16.0,4.0,14.0,1.0,37.0,1.0,1.0,0.0,182.0,139.0,3.68,1.237,8.2,0.8,2.9,7.6,2.64,L,BOS,AL,48.0,0.0,0.0,43.2,6.0,0.0,6.0,0.0,1.0,1.000,NaN,NaN,1.0,5.0,0.0,1.24,0.13,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BOS,8,48,0,4,48,48,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,,G,0,,0,0.0,,0.000000,,0,n,0,0.0,n,0,0.0,,0,66,G (n-n)
1,José Abreu,30,CHW,AL,156.0,675.0,621.0,95.0,189.0,43.0,6.0,33.0,102.0,3.0,0.0,35.0,119.0,0.304,0.354,0.552,0.906,141.0,343.0,21.0,15.0,0.0,4.0,6.0,*3D,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHW,AL,139.0,138.0,130.0,1197.0,1221.0,1135.0,78.0,8.0,130.0,0.993,3.0,3.0,2.0,2.0,-1.0,9.12,8.73,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,CHW,4,156,156,156,139,0,0,139,0,0,0,0,0,0,0,18,0,0,1B,0.653846,#,B+,6,16,1,7.0,(21),0.019108,,2,12,6,8.0,22,1,9.0,/23,10,53,#B+16(21) (12-22/23)
2,Cristhian Adames,25,COL,NL,12.0,14.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.000,0.071,0.000,0.071,-80.0,0.0,0.0,0.0,0.0,0.0,0.0,H/346,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COL,NL,3.0,1.0,0.0,9.0,4.0,3.0,1.0,0.0,0.0,1.000,-1.0,-93.0,-1.0,-133.0,0.0,4.00,1.33,SS-2B-1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,COL,4,12,1,12,2,0,0,1,1,0,1,0,0,0,0,0,10,0,SS,0.000000,,G,0,,0,0.0,,0.000000,,3,13,15,18.0,36,0,0.0,,0,66,G (13-36)
3,Austin Adams,26,WSN,NL,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,3.60,6.0,0.0,3.0,0.0,0.0,0.0,5.0,4.0,4.0,2.0,0.0,8.0,0.0,10.0,1.0,0.0,1.0,29.0,133.0,4.56,2.400,7.2,0.0,14.4,18.0,1.25,R,WSN,NL,6.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.000,NaN,NaN,0.0,0.0,0.0,1.80,0.17,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,WSN,1st,6,0,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,,G,0,,0,0.0,,0.000000,,0,n,0,0.0,n,0,0.0,,0,66,G (n-n)
4,Lane Adams,27,ATL,NL,85.0,122.0,109.0,19.0,30.0,4.0,1.0,5.0,20.0,10.0,0.0,10.0,37.0,0.275,0.339,0.468,0.807,109.0,51.0,3.0,1.0,1.0,1.0,0.0,H78/9,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NL,41.0,13.0,9.0,172.0,51.0,48.0,1.0,2.0,0.0,0.961,1.0,5.0,NaN,NaN,NaN,2.56,1.20,OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,ATL,2,85,13,85,41,0,0,0,0,0,0,27,11,7,41,0,51,9,OF,0.235294,,B,6,16,1,7.0,(21),0.322581,****,3,13,11,14.0,32,0,0.0,,9,54,B16(21)**** (13-32)
...,..

In [435]:
players.to_csv("../data/player stats - " + year + " - with batter ratings.csv", index=False)

## Pitcher Ratings

In [436]:
players = pd.read_csv("../data/player stats - " + year + " - with batter ratings.csv")

In [437]:
pd.set_option('display.max_seq_items', 150)
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B',
       ...
       'bb_num', 'k_rate', 'k_val', 'k_num', 'hbp_rate', 'hbp_val', 'hbp_num',
       'hit_rate', 'PH_num_bat', 'batter_rating'],
      dtype='object', length=155)

### Pitcher Letter Rating

In [438]:
players["BAA"] = round(players["H_pit"] /(players["BF"] - (players["BB_pit"] + players["HBP_pit"])),3)
players["BAA"].mean()

0.2647734627831717

In [439]:
baa_break_points = [
    0.140,
    0.168,
    0.196,
    0.223,
    0.251,
    0.279,
    0.307,
    0.335,
    0.362
]

letters = [
    "J+",
    "J",
    "K",
    "L",
    "M",
    "W",
    "X",
    "Y",
    "Z+",
    "Z"
]

def pitcher_letter(bat_avg_against, breakpoints=baa_break_points, letter_grades=letters):
    i = bisect(breakpoints, bat_avg_against)
    return letter_grades[i]

In [440]:
players["pit_letter"] = [pitcher_letter(avg) for avg in players["BAA"]]
players["pit_letter"].value_counts()

Z     786
W     205
M     200
X     122
L     120
Y      64
K      47
Z+     31
J+     28
J      28
Name: pit_letter, dtype: int64

In [441]:
players.loc[(players["IP"].isnull()), "pit_letter"] = ""
players["pit_letter"].value_counts()

      703
W     205
M     200
X     122
L     120
Z      83
Y      64
K      47
Z+     31
J+     28
J      28
Name: pit_letter, dtype: int64

In [442]:
players[players["pit_letter"] == "K"]["BAA"].min()

0.169

### Innings of Effectiveness Number

**NOTE** - IP is stored as .0, .1, .2 for full, one third, and two-thirds, so these need to be set to their true decimal values before any calculations using IP can be done.

In [443]:
players["IP_real"] = round(players["IP"]) + (10 * (players["IP"] - round(players["IP"])) / 3)
players["IP_real"].value_counts().head(25)

1.000000     29
5.000000     11
57.333333     9
2.000000      9
6.000000      8
10.333333     8
13.000000     8
4.000000      8
7.666667      8
27.000000     7
5.666667      7
20.000000     7
21.000000     7
8.000000      7
58.666667     7
14.333333     7
7.333333      7
0.666667      7
6.666667      7
8.333333      7
3.666667      7
55.666667     7
8.666667      6
62.000000     6
62.666667     6
Name: IP_real, dtype: int64

In [444]:
players["IE"] = round(players["IP_real"] / players ["G_pit"], 0)
players["IE"].value_counts()

1.0    514
5.0    111
6.0     97
2.0     73
3.0     57
4.0     55
0.0     15
7.0      6
Name: IE, dtype: int64

In [445]:
pd.set_option('display.max_columns', 160)
players[players["IE"] == 0]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE
127,Andres Blanco,33,PHI,NL,80.0,144.0,130.0,10.0,25.0,4.0,0.0,3.0,13.0,1.0,0.0,12.0,34.0,0.192,0.257,0.292,0.549,45.0,38.0,1.0,0.0,0.0,2.0,0.0,H543/6D1,S,NL,0.0,0.0,NaN,27.00,1.0,0.0,1.0,0.0,0.0,0.0,0.1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,33.0,42.16,3.000,27.0,27.0,0.0,0.0,NaN,R,PHI,NL,47.0,20.0,15.0,219.1,86.0,38.0,46.0,2.0,7.0,0.977,2.0,10.0,1.0,5.0,0.0,3.45,1.79,2B-3B-1B-SS-P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120,PHI,10,80,20,80,46,1,0,11,15,16,4,0,0,0,0,1,35,0,2B,0.162500,NaN,D+,4,14.0,0,0.0,NaN,0.033333,NaN,3,13,8,11.0,25,0,0.0,NaN,6,61,D+14 (13-25),0.500,Z,0.333333,0.0
284,Kyle Crockett,25,CLE,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,L,AL,0.0,0.0,NaN,10.80,4.0,0.0,0.0,0.0,0.0,0.0,1.2,4.0,2.0,2.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,11.0,51.0,4.36,3.000,21.6,0.0,5.4,10.8,2.00,L,CLE,AL,4.0,0.0,0.0,1.2,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.00,0.00,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,268,CLE,4,4,0,1,4,4,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.444,Z,1.666667,0.0
375,Jake Esch,27,SDP,NL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,R,SDP,NL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,353,SDP,2,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),NaN,Z,0.000000,0.0
709,Matt Koch,26,ARI,NL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,L,NL,0.0,0.0,NaN,inf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,R,ARI,NL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,661,ARI,2,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),1.000,Z,0.000000,0.0
1130,Andrew Romine,31,DET,AL,124.0,348.0,318.0,45.0,74.0,17.0,2.0,4.0,25.0,6.0,4.0,22.0,67.0,0.233,0.289,0.336,0.625,67.0,107.0,7.0,4.0,2.0,2.0,0.0,4853H796/D12,S,AL,0.0,0.0,NaN,0.00,2.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,6.0,NaN,9.16,2.000,9.0,0.0,9.0,0.0,0.00,R,DET,AL,136.0,79.0,61.0,752.2,305.0,210.0,92.0,3.0,24.0,0.990,-5.0,-7.0,-3.0,-5.0,0.0,3.61,2.22,OF-2B-SS-3B-1B-P-C,DET,AL,1.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,1041,DET,8,124,79,124

In [446]:
# Reset 0 Innings of Effectiveness to 1 (can't have 0 in SherCo)
players["IE"].replace(0, 1, inplace=True)
players["IE"].value_counts()

1.0    529
5.0    111
6.0     97
2.0     73
3.0     57
4.0     55
7.0      6
Name: IE, dtype: int64

In [447]:
players[players["IE"] >= 7]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE
176,Madison Bumgarner,27,SFG,NL,15.0,36.0,34.0,4.0,7.0,0.0,0.0,3.0,5.0,0.0,0.0,2.0,11.0,0.206,0.250,0.471,0.721,84.0,16.0,1.0,0.0,0.0,0.0,0.0,1,R,NL,4.0,9.0,0.308,3.32,17.0,17.0,0.0,1.0,0.0,0.0,111.0,101.0,41.0,41.0,17.0,20.0,3.0,101.0,3.0,0.0,0.0,450.0,128.0,3.95,1.090,8.2,1.4,1.6,8.2,5.05,L,SFG,NL,17.0,17.0,1.0,111.0,12.0,2.0,10.0,0.0,1.0,1.000,NaN,NaN,2.0,4.0,0.0,0.97,0.71,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168,SFG,9,17,17,15,17,17,0,0,0,0,0,0,0,0,0,0,0,0,P,0.333333,NaN,C,15,33.0,0,0.0,NaN,0.0,NaN,2,12,11,13.0,31,0,0.0,NaN,7,56,C33 (12-31),0.237,M,111.000000,7.0
448,Michael Fulmer,24,DET,AL,2.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.000,0.000,0.000,0.000,-100.0,0.0,1.0,0.0,0.0,0.0,0.0,1,R,AL,10.0,12.0,0.455,3.83,25.0,25.0,0.0,1.0,0.0,0.0,164.2,150.0,80.0,70.0,13.0,40.0,2.0,114.0,8.0,1.0,3.0,676.0,117.0,3.67,1.154,8.2,0.7,2.2,6.2,2.85,R,DET,AL,25.0,25.0,1.0,164.2,20.0,9.0,10.0,1.0,0.0,0.950,NaN,NaN,-2.0,-2.0,0.0,1.04,0.76,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,415,DET,2,25,25,2,25,25,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,14,14.0,32,0,0.0,NaN,0,66,G (n-32),0.239,M,164.666667,7.0
706,Corey Kluber,31,CLE,AL,1.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500,0.500,0.500,1.000,164.0,1.0,0.0,0.0,0.0,0.0,0.0,1,R,AL,18.0,4.0,0.818,2.25,29.0,29.0,0.0,5.0,3.0,0.0,203.2,141.0,56.0,51.0,21.0,36.0,2.0,265.0,5.0,0.0,4.0,777.0,202.0,2.50,0.869,6.2,0.9,1.6,11.7,7.36,R,CLE,AL,29.0,29.0,5.0,203.2,32.0,15.0,16.0,1.0,1.0,0.969,NaN,NaN,-1.0,-1.0,0.0,1.37,1.07,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,658,CLE,7,29,29,1,29,29,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,AAA,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,18,41,AAA (n-n),0.192,K,203.666667,7.0
1166,Chris Sale,28,BOS,AL,1.0,3.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.333,0.333,0.667,1.000,152.0,2.0,0.0,0.0,0.0,0.0,0.0,1,L,AL,17.0,8.0,0.680,2.90,32.0,32.0,0.0,1.0,0.0,0.0,214.1,165.0,73.0,69.0,24.0,43.0,0.0,308.0,8.0,0.0,3.0,851.0,157.0,2.45,0.970,6.9,1.0,1.8,12.9,7.16,L,BOS,AL,32.0,32.0,1.0,214.1,30.0,8.0,22.0,0.0,3.0,1.000,NaN,NaN,2.0,2.0,0.0,1.26,0.94,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1075,BOS,8,32,32,1,32,32,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,A,0,NaN,0,0.0,NaN,0.0,NaN,0,n,12,12.0,26,0,0.0,NaN,12,51,A (n-26),0.206,L,214.333333,7.0
1504,Jaime Garcia,30,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,1.0,0.0,1.000,4.05,1.0,1.0,0.0,0.0,0.0,0.0,6.2,8.0,3.0,3.0,0.0,3.0,0.0,7.0,0.0,0.0,0.0,29.0,115.0,2.41,1.650,10.8,0.0,4.1,9.5,2.33,L,TOT,ZZ,27.0,27.0,0.0,157.0,34.0,4.0,28.0,2.0,2.0,0.941,NaN,NaN,NaN,NaN,NaN,1.83,1.19,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [448]:
players["IE"] = players["IE"].astype('Int64')

### Base on Balls Number

In [449]:
players["bb_rate"] = round(players["BB_pit"] / players["BF"] * 36, 0)
players["bb_rate"].replace(np.nan, 0, inplace=True)
players["bb_rate"] = players["bb_rate"].astype(int)
players["bb_rate"].value_counts()

0     745
3     301
4     200
2     187
5      80
6      35
1      34
7      15
9      10
8       8
12      7
10      4
24      1
14      1
18      1
21      1
36      1
Name: bb_rate, dtype: int64

In [450]:
players.loc[(players["BF"].isnull()), "bb_rate"] = np.nan
players["bb_rate"].value_counts()

3.0     301
4.0     200
2.0     187
5.0      80
0.0      42
6.0      35
1.0      34
7.0      15
9.0      10
8.0       8
12.0      7
10.0      4
18.0      1
36.0      1
21.0      1
14.0      1
24.0      1
Name: bb_rate, dtype: int64

In [451]:
players["bb_num_pit"] = players["bb_rate"].map({
    0: "11",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["bb_num_pit"].value_counts()

nan    703
13     301
14     200
12     187
15      80
11      76
16      35
21      15
23      10
22       8
26       7
24       4
32       1
43       1
36       1
46       1
66       1
Name: bb_num_pit, dtype: int64

### Strikeout Number

In [452]:
players["k_rate_pit"] = round(players["SO_pit"] / players["BF"] * 36, 0)
players["k_rate_pit"].value_counts()

7.0     158
8.0     144
6.0     123
9.0     106
10.0     89
5.0      85
4.0      46
0.0      38
11.0     37
12.0     32
3.0      21
2.0      14
13.0     13
14.0     10
15.0      6
18.0      3
1.0       2
16.0      1
Name: k_rate_pit, dtype: int64

In [453]:
players[players["k_rate_pit"]==0]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit
46,Nori Aoki,35,TOT,MLB,109.0,374.0,336.0,48.0,93.0,20.0,2.0,5.0,35.0,10.0,2.0,29.0,44.0,0.277,0.335,0.393,0.728,98.0,132.0,12.0,3.0,1.0,5.0,1.0,79H/8D1,L,AL,0.0,0.0,NaN,27.00,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6.0,26.0,9.16,3.000,9.0,0.0,18.0,0.0,0.0,R,TOT,MLB,105.0,87.0,56.0,766.0,141.0,133.0,7.0,1.0,1.0,0.993,-11.0,-18.0,NaN,0.0,NaN,1.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,3TM,6,110,87,109,105,1,0,0,0,0,0,64,2,47,104,1,10,1,OF,0.321101,NaN,B,2,12.0,1,3.0,(13),0.102041,**,3,13,4,7.0,21,0,0.0,NaN,9,54,B12(13)** (13-21),0.250,M,1.000000,1,12.0,26,0.0
47,Nori Aoki,35,TOT,AL,82.0,258.0,234.0,32.0,64.0,13.0,1.0,5.0,27.0,5.0,2.0,16.0,34.0,0.274,0.319,0.402,0.721,96.0,94.0,11.0,2.0,1.0,5.0,1.0,79/H8D1,L,AL,0.0,0.0,NaN,27.00,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6.0,26.0,9.16,3.000,9.0,0.0,18.0,0.0,0.0,R,TOT,MLB,105.0,87.0,56.0,766.0,141.0,133.0,7.0,1.0,1.0,0.993,-11.0,-18.0,NaN,0.0,NaN,1.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,3TM,6,110,87,109,105,1,0,0,0,0,0,64,2,47,104,1,10,1,OF,0.329268,NaN,B,3,13.0,1,4.0,(14),0.079365,*,2,12,5,7.0,21,0,0.0,NaN,9,54,B13(14)* (12-21),0.250,M,1.000000,1,12.0,26,0.0
48,Nori Aoki,35,HOU,AL,70.0,224.0,202.0,28.0,55.0,12.0,1.0,2.0,19.0,5.0,2.0,15.0,29.0,0.272,0.323,0.371,0.694,91.0,75.0,11.0,2.0,1.0,4.0,1.0,79/HD1,L,AL,0.0,0.0,NaN,27.00,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6.0,21.0,9.16,3.000,9.0,0.0,18.0,0.0,0.0,R,TOT,MLB,105.0,87.0,56.0,766.0,141.0,133.0,7.0,1.0,1.0,0.993,-11.0,-18.0,NaN,0.0,NaN,1.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,3TM,6,110,87,109,105,1,0,0,0,0,0,64,2,47,104,1,10,1,OF,0.271429,NaN,B,1,11.0,1,2.0,(12),0.087719,*,2,12,5,7.0,21,0,0.0,NaN,9,54,B11(12)* (12-21),0.250,M,1.000000,1,12.0,26,0.0
71,Erick Aybar,33,SDP,NL,108.0,370.0,333.0,37.0,78.0,15.0,1.0,7.0,22.0,11.0,4.0,28.0,57.0,0.234,0.300,0.348,0.648,73.0,116.0,5.0,4.0,3.0,2.0,2.0,6H/1,S,NL,0.0,0.0,NaN,0.00,2.0,0.0,2.0,0.0,0.0,0.0,1.1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,NaN,5.41,0.750,0.0,0.0,6.8,0.0,0.0,R,SDP,NL,101.0,93.0,79.0,794.1,399.0,117.0,273.0,9.0,63.0,0.977,-6.0,-9.0,-9.0,-2.0,-1.0,4.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,SDP,12,108,93,108,100,2,0,0,0,0,99,0,0,0,0,0,12,0,SS,0.203704,NaN,C+,3,13.0,0,0.0,NaN,0.126437,**,3,13,6,9.0,23,0,0.0,NaN,8,55,C+13** (13-23),0.000,J+,1.333333,1,9.0,23,0.0
127,Andres Blanco,33,PHI,NL,80.0,144.0,130.0,10.0,25.0,4.0,0.0,3.0,13.0,1.0,0.0,12.0,34.0,0.192,0.257,0.292,0.549,45.0,38.0,1.0,0.0,0.0,2.0,0.0,H543/6D1,S,NL,0.0,0.0,NaN,27.00,1.0,0.0,1.0,0.0,0.0,0.0,0.1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,33.0,42.16,3.000,27.0,27.0,0.0,0.0,NaN,R,PHI,NL,47.0,20.0,15.0,219.

In [454]:
players.loc[(players["k_rate_pit"] == 0), "k_val_pit"] = 0
players.loc[(players["k_rate_pit"] > 0), "k_val_pit"] = players["bb_rate"] + players["k_rate_pit"]
players["k_val_pit"].value_counts()

11.0    147
10.0    133
12.0    112
9.0      93
8.0      87
13.0     83
7.0      51
14.0     49
15.0     45
0.0      38
6.0      19
17.0     18
16.0     16
18.0     11
5.0      10
20.0      4
4.0       4
36.0      2
1.0       1
21.0      1
19.0      1
26.0      1
23.0      1
22.0      1
Name: k_val_pit, dtype: int64

In [455]:
players["k_num_pit"] = players["k_val_pit"].map({
    0: "n",
    1: "11",
    2: "12",
    3: "13",
    4: "14",
    5: "15",
    6: "16",
    7: "21",
    8: "22",
    9: "23",
    10: "24",
    11: "25",
    12: "26",
    13: "31",
    14: "32",
    15: "33",
    16: "34",
    17: "35",
    18: "36",
    19: "41",
    20: "42",
    21: "43",
    22: "44",
    23: "45",
    24: "46",
    25: "51",
    26: "52",
    27: "53",
    28: "54",
    29: "55",
    30: "56",
    31: "61",
    32: "62",
    33: "63",
    34: "64",
    35: "65",
    36: "66"
}).astype(str)
players["k_num_pit"].value_counts()

nan    703
25     147
24     133
26     112
23      93
22      87
31      83
21      51
32      49
33      45
n       38
16      19
35      18
34      16
36      11
15      10
42       4
14       4
66       2
43       1
52       1
41       1
45       1
11       1
44       1
Name: k_num_pit, dtype: int64

In [456]:
players[players["Name"] == "Dwight\xa0Gooden"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit


### Hit Batter Number

In [457]:
players["hbp_rate_pit"] = round(players["HBP_pit"] / players["BF"] * 36, 0)
players["hbp_rate_pit"].replace(np.nan, 0, inplace=True)
players["hbp_rate_pit"] = players["hbp_rate_pit"].astype(int)
players["hbp_rate_pit"].value_counts()

0    1403
1     199
2      17
3       6
6       2
4       2
7       1
5       1
Name: hbp_rate_pit, dtype: int64

In [458]:
players.loc[(players["hbp_rate_pit"] == 0), "hbp_val_pit"] = 0
players.loc[(players["hbp_rate_pit"] > 0), "hbp_val_pit"] = players["k_val_pit"] + players["hbp_rate_pit"]

In [459]:
players["hbp_val_pit"].value_counts()

0.0     1403
11.0      44
12.0      38
10.0      27
9.0       24
13.0      20
14.0      16
15.0      15
16.0      13
8.0        9
7.0        7
6.0        3
18.0       3
19.0       2
17.0       2
22.0       1
21.0       1
24.0       1
5.0        1
23.0       1
Name: hbp_val_pit, dtype: int64

In [460]:
players["hbp_num_pit"] = players["hbp_val_pit"].map({
    0: "",
    1: "/11",
    2: "/12",
    3: "/13",
    4: "/14",
    5: "/15",
    6: "/16",
    7: "/21",
    8: "/22",
    9: "/23",
    10: "/24",
    11: "/25",
    12: "/26",
    13: "/31",
    14: "/32",
    15: "/33",
    16: "/34",
    17: "/35",
    18: "/36",
    19: "/41",
    20: "/42",
    21: "/43",
    22: "/44",
    23: "/45",
    24: "/46",
    25: "/51",
    26: "/52",
    27: "/53",
    28: "/54",
    29: "/55",
    30: "/56",
    31: "/61",
    32: "/62",
    33: "/63",
    34: "/64",
    35: "/65",
    36: "/66"
}).astype(str)
players["hbp_num_pit"].value_counts()

       1403
/25      44
/26      38
/24      27
/23      24
/31      20
/32      16
/33      15
/34      13
/22       9
/21       7
/36       3
/16       3
/41       2
/35       2
/15       1
/45       1
/46       1
/43       1
/44       1
Name: hbp_num_pit, dtype: int64

### Wild Pitch Rating

In [461]:
pd.set_option('display.max_seq_items', 200)
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'Rdrs', 'Rdrs/yr', 'Rgood', 'RF/9',
       'RF/G', 'Pos_Summary_fld', 'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat',
       'CG_cat', 'Inn_cat', 'Ch_cat', 'PO_cat', 'A_cat', 'E_cat', 'DP_cat',
       'Fld%_cat', 'Rtot_cat', 'Rtot/yr_cat', 'Rctch', 'Rdrs_cat',
       'Rdrs/yr_cat', 'Rgood_cat', 'Rair', 'Rrange', 'Rthrow', 'RszC', 'RsbC',
       'RerC', 'RF/9_cat

In [462]:
players["WP"].value_counts()

0.0     298
1.0     197
2.0     129
3.0      88
4.0      68
5.0      45
6.0      30
7.0      26
8.0      16
9.0       7
11.0      7
10.0      5
12.0      5
14.0      2
15.0      2
13.0      2
16.0      1
Name: WP, dtype: int64

In [463]:
players[players["WP"] >= 10]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit
53,Chris Archer,28,TBR,AL,3.0,8.0,8.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.250,0.250,0.250,0.500,38.0,2.0,1.0,0.0,0.0,0.0,0.0,1,R,AL,10.0,12.0,0.455,4.07,34.0,34.0,0.0,0.0,0.0,0.0,201.0,193.0,101.0,91.0,27.0,60.0,0.0,249.0,5.0,0.0,15.0,852.0,103.0,3.40,1.259,8.6,1.2,2.7,11.1,4.15,R,TBR,AL,34.0,34.0,0.0,201.0,15.0,8.0,6.0,1.0,0.0,0.933,NaN,NaN,-3.0,-3.0,0.0,0.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,TBR,6,34,34,3,34,34,0,0,0,0,0,0,0,0,0,0,0,0,P,0.333333,NaN,C+,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,14,14.0,32,0,0.0,NaN,9,54,C+ (n-32),0.245,M,201.000000,6,3.0,13,11.0,14.0,32,0,0.0,
57,Jake Arrieta,31,CHC,NL,29.0,64.0,61.0,2.0,8.0,0.0,1.0,1.0,5.0,0.0,0.0,1.0,32.0,0.131,0.145,0.213,0.358,-9.0,13.0,0.0,0.0,2.0,0.0,0.0,1/H,R,NL,14.0,10.0,0.583,3.53,30.0,30.0,0.0,0.0,0.0,0.0,168.1,150.0,82.0,66.0,23.0,55.0,3.0,163.0,10.0,0.0,14.0,707.0,124.0,4.16,1.218,8.0,1.2,2.9,8.7,2.96,R,CHC,NL,30.0,30.0,0.0,168.1,43.0,15.0,26.0,2.0,2.0,0.953,NaN,NaN,-3.0,-4.0,-1.0,2.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54,CHC,8,32,30,29,30,30,0,0,0,0,0,0,0,0,0,0,2,0,P,0.172414,NaN,E+,4,14.0,4,8.0,(22),0.000000,NaN,1,11,18,19.0,41,0,0.0,NaN,4,63,E+14(22) (11-41),0.234,M,168.333333,6,3.0,13,8.0,11.0,25,1,12.0,/26
189,Trevor Cahill,29,TOT,MLB,10.0,18.0,14.0,1.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,4.0,0.286,0.333,0.357,0.690,86.0,5.0,0.0,0.0,3.0,0.0,0.0,1,R,MLB,4.0,3.0,0.571,4.93,21.0,14.0,1.0,0.0,0.0,0.0,84.0,91.0,50.0,46.0,16.0,45.0,1.0,87.0,3.0,0.0,16.0,381.0,88.0,5.28,1.619,9.8,1.7,4.8,9.3,1.93,R,TOT,ZZ,21.0,14.0,0.0,84.0,23.0,3.0,20.0,0.0,3.0,1.000,NaN,NaN,NaN,NaN,NaN,2.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180,2TM,9,21,14,10,21,21,0,0,0,0,0,0,0,0,0,0,0,0,P,0.200000,NaN,B+,0,NaN,0,0.0,NaN,0.000000,NaN,2,12,8,10.0,24,0,0.0,NaN,8,55,B+ (12-24),0.273,W,84.000000,4,4.0,14,8.0,12.0,26,0,0.0,
190,Trevor Cahill,29,SDP,NL,9.0,17.0,13.0,1.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,3.0,0.308,0.357,0.385,0.742,99.0,5.0,0.0,0.0,3.0,0.0,0.0,1,R,NL,4.0,3.0,0.571,3.69,11.0,11.0,0.0,0.0,0.0,0.0,61.0,58.0,29.0,25.0,6.0,24.0,1.0,72.0,3.0,0.0,14.0,263.0,115.0,3.40,1.344,8.6,0.9,3.5,10.6,3.00,R,TOT,ZZ,21.0,14.0,0.0,84.0,23.0,3.0,20.0,0.0,3.0,1.000,NaN,NaN,NaN,NaN,NaN,2.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180,2TM,9,21,14,10,21,21,0,0,0,0,0,0,0,0,0,0,0,0,P,0.222222,NaN,A,0,NaN,0,0.0,NaN,0.000000,NaN,2,12,6,8.0,22,0,0.0,NaN,8,55,A (12-22),0.246,M,61.000000,6,3.0,13,10.0,13.0,31,0,0.0,
209,Carlos Carrasco,30,CLE,AL,2.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000,0.200,0.000,0.200,-40.0,0.0,0.0,0.0,0.0,0.0,0.0,1,R,AL,18.0,6.0,0.750,3.29,32.0,32.0,0.0,1.0,0.0,0.0,200.0,173.0,73.0,73.0,21.0,46.0,2.0,226.0,10.0,0.0,10.0,798.0,139.0,3.10,1.095,7.8,0.9,2.1,10.2,4.91,R,CLE,AL,32.0,32.0,1.0,200.0,21.0,4.0,17.0,0.0,1.0,1.000,NaN,NaN,2.0,2

In [464]:
players.loc[(players["WP"] < 5), "WP_num"] = ""
players.loc[(players["WP"] >= 5), "WP_num"] = "[WP]"
players["WP_num"].value_counts()

        780
[WP]    148
Name: WP_num, dtype: int64

### Gopher Ball Rating

In [465]:
players["hr_rate_pit"] = players["HR_pit"] / players["H_pit"]
players["hr_rate_pit"].value_counts()

0.000000    92
0.142857    29
0.166667    25
0.250000    22
0.200000    20
            ..
0.158824     1
0.264286     1
0.114035     1
0.132911     1
0.318182     1
Name: hr_rate_pit, Length: 356, dtype: int64

In [466]:
players["gopher_ball"] = ""
players.loc[(players["hr_rate_pit"] >= .1), "gopher_ball"] = "+"
players.loc[(players["hr_rate_pit"] <= .05), "gopher_ball"] = "-"
players["gopher_ball"].value_counts()

     855
+    662
-    114
Name: gopher_ball, dtype: int64

In [467]:
players[players["gopher_ball"] == "-"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball
3,Austin Adams,26,WSN,NL,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,3.60,6.0,0.0,3.0,0.0,0.0,0.0,5.0,4.0,4.0,2.0,0.0,8.0,0.0,10.0,1.0,0.0,1.0,29.0,133.0,4.56,2.400,7.2,0.0,14.4,18.0,1.25,R,WSN,NL,6.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.000,NaN,NaN,0.0,0.0,0.0,1.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,WSN,1st,6,0,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,P,0.0,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.200,L,5.000000,1,10.0,24,12.0,22.0,44,1,23.0,/45,,0.000000,-
16,Al Alburquerque,31,TOT,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,R,AL,0.0,2.0,0.000,2.50,21.0,0.0,7.0,0.0,0.0,0.0,18.0,10.0,5.0,5.0,0.0,8.0,0.0,14.0,0.0,0.0,0.0,71.0,182.0,2.94,1.000,5.0,0.0,4.0,7.0,1.75,R,TOT,AL,21.0,0.0,0.0,18.0,9.0,2.0,7.0,0.0,0.0,1.000,NaN,NaN,NaN,NaN,NaN,4.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2TM,7,21,0,1,21,21,0,0,0,0,0,0,0,0,0,0,0,0,P,0.0,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.159,J,18.000000,1,4.0,14,7.0,11.0,25,0,0.0,,,0.000000,-
17,Al Alburquerque,31,KCR,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,R,AL,0.0,1.0,0.000,3.60,11.0,0.0,5.0,0.0,0.0,0.0,10.0,7.0,4.0,4.0,0.0,6.0,0.0,9.0,0.0,0.0,0.0,42.0,128.0,3.16,1.300,6.3,0.0,5.4,8.1,1.50,R,TOT,AL,21.0,0.0,0.0,18.0,9.0,2.0,7.0,0.0,0.0,1.000,NaN,NaN,NaN,NaN,NaN,4.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2TM,7,21,0,1,21,21,0,0,0,0,0,0,0,0,0,0,0,0,P,0.0,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.194,K,10.000000,1,5.0,15,8.0,13.0,31,0,0.0,,,0.000000,-
20,Scott Alexander,27,KCR,AL,4.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,1,L,AL,5.0,4.0,0.556,2.48,58.0,0.0,9.0,0.0,0.0,4.0,69.0,62.0,23.0,19.0,3.0,28.0,0.0,59.0,0.0,0.0,3.0,283.0,181.0,3.23,1.304,8.1,0.4,3.7,7.7,2.11,L,KCR,AL,58.0,0.0,0.0,69.0,15.0,6.0,7.0,2.0,0.0,0.867,NaN,NaN,0.0,0.0,0.0,1.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,KCR,3,58,0,4,58,58,0,0,0,0,0,0,0,0,0,0,0,0,P,0.0,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,12,12.0,26,0,0.0,NaN,0,66,G (n-26),0.243,M,69.000000,1,4.0,14,8.0,12.0,26,0,0.0,,,0.048387,-
41,Drew Anderson,23,PHI,NL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,23.14,2.0,0.0,1.0,0.0,0.0,0.0,2.1,6.0,7.0,6.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,14.0,21.0,2.73,3.000,23.1,0.0,3.9,7.7,2.00,R,PHI,NL,2.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,-1.0,-86.0,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,PHI,1st,2,0,1,2,2,0,0,0,0,0,0,0,0,0,0,0,0,P,0.0,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0

### Pitcher Control Number

In [468]:
players["control_rate"] = round((players["BB_pit"] + players["HBP_pit"] + players["H_pit"]) / 
                                players["BF"] * 36, 0)
players["control_rate"].value_counts()

12.0    197
11.0    170
13.0    139
10.0    106
14.0     80
9.0      45
15.0     39
16.0     33
8.0      25
17.0     20
18.0     16
19.0     11
20.0      8
7.0       8
0.0       7
22.0      5
21.0      4
6.0       3
36.0      2
30.0      2
24.0      2
29.0      2
5.0       1
3.0       1
26.0      1
4.0       1
Name: control_rate, dtype: int64

In [469]:
players["PCN"] = players["control_rate"].map({
    0: "65",
    1: "65",
    2: "64",
    3: "63",
    4: "62",
    5: "61",
    6: "56",
    7: "55",
    8: "54",
    9: "53",
    10: "52",
    11: "51",
    12: "46",
    13: "45",
    14: "44",
    15: "43",
    16: "42",
    17: "41",
    18: "36",
    19: "35",
    20: "34",
    21: "33",
    22: "32",
    23: "31",
    24: "26",
    25: "25",
    26: "24",
    27: "23",
    28: "22",
    29: "21",
    30: "16",
    31: "15",
    32: "14",
    33: "13",
    34: "12",
    35: "11",
    36: "11"
}).astype(str)
players["PCN"].value_counts()

nan    703
46     197
51     170
45     139
52     106
44      80
53      45
43      39
42      33
54      25
41      20
36      16
35      11
55       8
34       8
65       7
32       5
33       4
56       3
16       2
21       2
26       2
11       2
62       1
61       1
63       1
24       1
Name: PCN, dtype: int64

### Probable Hit Number

In [470]:
players["hit_rate_pit"] = round(players["H_pit"] / players["BF"] * 36, 0)
players["hit_rate_pit"].replace(np.nan, 0, inplace=True)
players["hit_rate_pit"] = players["hit_rate_pit"].astype(int)
players["hit_rate_pit"].value_counts()

0     717
8     233
9     192
7     150
10     87
6      74
11     47
5      32
14     22
12     22
13     12
4      12
15      8
18      5
22      4
3       4
16      3
17      3
24      1
20      1
2       1
27      1
Name: hit_rate_pit, dtype: int64

In [471]:
players.loc[(players["BF"].isnull()), "hit_rate_pit"] = np.nan

In [472]:
players["hit_rate_pit"].value_counts()

8.0     233
9.0     192
7.0     150
10.0     87
6.0      74
11.0     47
5.0      32
14.0     22
12.0     22
0.0      14
4.0      12
13.0     12
15.0      8
18.0      5
3.0       4
22.0      4
16.0      3
17.0      3
20.0      1
2.0       1
24.0      1
27.0      1
Name: hit_rate_pit, dtype: int64

In [473]:
players["PPH"] = players["hit_rate_pit"].map({
    0: "66",
    1: "66",
    2: "65",
    3: "64",
    4: "63",
    5: "62",
    6: "61",
    7: "56",
    8: "55",
    9: "54",
    10: "53",
    11: "52",
    12: "51",
    13: "46",
    14: "45",
    15: "44",
    16: "43",
    17: "42",
    18: "41",
    19: "36",
    20: "35",
    21: "34",
    22: "33",
    23: "32",
    24: "31",
    25: "26",
    26: "25",
    27: "24",
    28: "23",
    29: "22",
    30: "21",
    31: "16",
    32: "15",
    33: "14",
    34: "13",
    35: "12",
    36: "11"
}).astype(str)
players["PPH"].value_counts()

nan    703
55     233
54     192
56     150
53      87
61      74
52      47
62      32
51      22
45      22
66      14
46      12
63      12
44       8
41       5
64       4
33       4
43       3
42       3
35       1
65       1
24       1
31       1
Name: PPH, dtype: int64

### Pitcher Rating

In [474]:
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'Rdrs', 'Rdrs/yr', 'Rgood', 'RF/9',
       'RF/G', 'Pos_Summary_fld', 'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat',
       'CG_cat', 'Inn_cat', 'Ch_cat', 'PO_cat', 'A_cat', 'E_cat', 'DP_cat',
       'Fld%_cat', 'Rtot_cat', 'Rtot/yr_cat', 'Rctch', 'Rdrs_cat',
       'Rdrs/yr_cat', 'Rgood_cat', 'Rair', 'Rrange', 'Rthrow', 'RszC', 'RsbC',
       'RerC', 'RF/9_cat

In [475]:
players["goph_lett_inn"] = players["gopher_ball"] + players["pit_letter"] + players["IE"].astype(str)
players.loc[(players["IP"].isnull()), "goph_lett_inn"] = ""
players["goph_lett_inn"].value_counts()

       703
+M1     73
+W1     66
+L1     51
+W5     40
      ... 
Z2       1
J1       1
M4       1
X4       1
+M7      1
Name: goph_lett_inn, Length: 115, dtype: int64

In [476]:
players["bb_k_hbp"] = "(" + players["bb_num_pit"] + "-" + players["k_num_pit"] + players["hbp_num_pit"] + ") "
players["bb_k_hbp"].value_counts()

(nan-nan)      703
(13-25)         50
(13-24)         47
(14-25)         31
(13-26)         30
              ... 
(12-23/25)       1
(22-33)          1
(23-32)          1
(15-33/34)       1
(12-42/43)       1
Name: bb_k_hbp, Length: 188, dtype: int64

In [477]:
players.loc[(players["IP"].isnull()), "bb_k_hbp"] = ""
players["bb_k_hbp"].value_counts()

               703
(13-25)         50
(13-24)         47
(14-25)         31
(13-26)         30
              ... 
(12-23/25)       1
(22-33)          1
(23-32)          1
(15-33/34)       1
(46-66)          1
Name: bb_k_hbp, Length: 188, dtype: int64

In [478]:
players["pitcher_rating"] = players["goph_lett_inn"] + " " + players["bb_k_hbp"] + " " + players["WP_num"]
players["pitcher_rating"].value_counts()

+M1 (13-26)         7
+W5 (13-23)         5
-M1 (11-n)          5
+M1 (13-31)         5
+M6 (13-25)         5
                   ..
+M6 (12-22)         1
J1 (13-35/36)       1
+J2 (12-15/22)      1
W1 (12-21)          1
+J1 (14-35)         1
Name: pitcher_rating, Length: 755, dtype: int64

In [479]:
players[players["Name"] == "Dwight\xa0Gooden"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating


In [480]:
players.head(10)

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating
0,Fernando Abad,31,BOS,AL,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,L,AL,2.0,1.0,0.667,3.30,48.0,0.0,15.0,0.0,0.0,1.0,43.2,40.0,18.0,16.0,4.0,14.0,1.0,37.0,1.0,1.0,0.0,182.0,139.0,3.68,1.237,8.2,0.8,2.9,7.6,2.64,L,BOS,AL,48.0,0.0,0.0,43.2,6.0,0.0,6.0,0.0,1.0,1.000,NaN,NaN,1.0,5.0,0.0,1.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BOS,8,48,0,4,48,48,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.240,M,43.666667,1,3.0,13,7.0,10.0,24,0,0.0,,,0.100000,+,11.0,51,8.0,55,+M1,(13-24),+M1 (13-24)
1,José Abreu,30,CHW,AL,156.0,675.0,621.0,95.0,189.0,43.0,6.0,33.0,102.0,3.0,0.0,35.0,119.0,0.304,0.354,0.552,0.906,141.0,343.0,21.0,15.0,0.0,4.0,6.0,*3D,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHW,AL,139.0,138.0,130.0,1197.0,1221.0,1135.0,78.0,8.0,130.0,0.993,3.0,3.0,2.0,2.0,-1.0,9.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,CHW,4,156,156,156,139,0,0,139,0,0,0,0,0,0,0,18,0,0,1B,0.653846,#,B+,6,16.0,1,7.0,(21),0.019108,NaN,2,12,6,8.0,22,1,9.0,/23,10,53,#B+16(21) (12-22/23),NaN,,NaN,<NA>,NaN,nan,NaN,NaN,nan,0,0.0,,NaN,NaN,,NaN,nan,NaN,nan,,,NaN
2,Cristhian Adames,25,COL,NL,12.0,14.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.000,0.071,0.000,0.071,-80.0,0.0,0.0,0.0,0.0,0.0,0.0,H/346,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COL,NL,3.0,1.0,0.0,9.0,4.0,3.0,1.0,0.0,0.0,1.000,-1.0,-93.0,-1.0,-133.0,0.0,4.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,COL,4,12,1,12,2,0,0,1,1,0,1,0,0,0,0,0,10,0,SS,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000000,NaN,3,13,15,18.0,36,0,0.0,NaN,0,66,G (13-36),NaN,,NaN,<NA>,NaN,nan,NaN,NaN,nan,0,0.0,,NaN,NaN,,NaN,nan,NaN,nan,,,NaN
3,Austin Adams,26,WSN,NL,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,3.60,6.0,0.0,3.0,0.0,0.0,0.0,5.0,4.0,4.0,2.0,0.0,8.0,0.0,10.0,1.0,0.0,1.0,29.0,133.0,4.56,2.400,7.2,0.0,14.4,18.0,1.25,R,WSN,NL,6.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.000,NaN,NaN,0.0,0.0,0.0,1.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,WSN,1st,6,0,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.200,L,5.000000,1,10.0,24,12.0,22.0,44,1,23.0,/45,,0.000000,-,16.0,42,5.0,62,-L1,(24-44/45),-L1 (24-44/45)
4,Lane Adams,27,ATL,NL,85.0,122.0,109.0,19.0,30.0,4.0,1.0,5.0,20.0,10.0,0.0,10.0,37.0,0.275,0.339,0.468,0.807,109.0,51.0,3.0,1.0,1.0,1.0,0.0,H78/9,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NL,41.0,13.0,9.0,172.0,51.0,48.0,1.0,2.0,0.0,0.961,1.0,5.0,NaN,NaN,NaN,2.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [481]:
players.tail(10)

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,Rdrs_cat,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating
1621,Tyler Webb,26,NYY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,0.0,NaN,4.50,7.0,0.0,2.0,0.0,0.0,0.0,6.0,3.0,3.0,3.0,1.0,4.0,0.0,5.0,0.0,0.0,0.0,23.0,106.0,5.66,1.167,4.5,1.5,6.0,7.5,1.25,L,TOT,ZZ,9.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1302,2TM,1st,9,0,2,9,9,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.158,J,6.000000,1,6.0,16,8.0,14.0,32,0,0.0,,,0.333333,+,11.0,51,5.0,62,+J1,(16-32),+J1 (16-32)
1622,Ryan Weber,26,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,0.0,NaN,2.45,1.0,1.0,0.0,0.0,0.0,0.0,3.2,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,186.0,3.16,0.818,7.4,0.0,0.0,0.0,NaN,R,SEA,AL,1.0,1.0,0.0,3.2,2.0,1.0,1.0,0.0,0.0,1.000,NaN,NaN,0.0,0.0,0.0,4.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1303,SEA,3,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.214,L,3.666667,4,0.0,11,0.0,0.0,n,0,0.0,,,0.000000,-,8.0,54,8.0,55,-L4,(11-n),-L4 (11-n)
1623,Rob Whalen,23,SEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,1.0,0.00,6.14,2.0,1.0,1.0,0.0,0.0,0.0,7.1,7.0,5.0,5.0,1.0,2.0,0.0,2.0,2.0,0.0,0.0,31.0,71.0,6.02,1.227,8.6,1.2,2.5,2.5,1.00,R,SEA,AL,2.0,1.0,0.0,7.1,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1307,SEA,2,2,1,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.259,W,7.333333,4,2.0,12,2.0,4.0,14,2,6.0,/16,,0.142857,+,13.0,45,8.0,55,+W4,(12-14/16),+W4 (12-14/16)
1624,Jason Wheeler,26,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,0.0,NaN,9.00,2.0,0.0,1.0,0.0,0.0,0.0,3.0,6.0,5.0,3.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,18.0,55.0,11.49,3.333,18.0,3.0,12.0,0.0,0.00,L,MIN,AL,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1308,MIN,1st,2,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.429,Z,3.000000,2,8.0,22,0.0,0.0,n,0,0.0,,,0.166667,+,20.0,34,12.0,51,+Z2,(22-n),+Z2 (22-n)
1625,Adam Wilk,29,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,0.0,1.0,0.00,7.84,3.0,1.0,0.0,0.0,0.0,0.0,10.1,16.0,9.0,9.0,3.0,8.0,1.0,6.0,0.0,0.0,2.0,54.0,58.0,8.09,2.323,13.9,2.6,7.0,5.2,0.75,L,TOT,ZZ,4.0,2.0,0.0,14.0,7.0,5.0,2.0,0.0,0.0,1.000,NaN,NaN,NaN,NaN,NaN,4.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1314,2TM,4,4,2,1,4,4,0,0,0,0,0,0,0,0,0,0,0,0,P,NaN,NaN,G,0,NaN,0,0.0,NaN,0.0,NaN,0,n,0,0.0,n,0,0.0,NaN

In [482]:
players.to_csv("../data/player stats - " + year + " - with batter and pitcher ratings.csv", index=False)

## Fielding Ratings

In [483]:
players = pd.read_csv("../data/player stats - " + year + " - with batter and pitcher ratings.csv")

In [484]:
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'Rdrs', 'Rdrs/yr', 'Rgood', 'RF/9',
       'RF/G', 'Pos_Summary_fld', 'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat',
       'CG_cat', 'Inn_cat', 'Ch_cat', 'PO_cat', 'A_cat', 'E_cat', 'DP_cat',
       'Fld%_cat', 'Rtot_cat', 'Rtot/yr_cat', 'Rctch', 'Rdrs_cat',
       'Rdrs/yr_cat', 'Rgood_cat', 'Rair', 'Rrange', 'Rthrow', 'RszC', 'RsbC',
       'RerC', 'RF/9_cat

In [485]:
players["Primary_Pos_fld"].value_counts()

P     903
OF    284
C     118
2B     88
3B     83
SS     77
1B     74
Name: Primary_Pos_fld, dtype: int64

In [486]:
players["Primary_Pos_fld"].isnull().sum()

4

In [487]:
players.groupby("Primary_Pos_fld")["Fld%"].mean()

Primary_Pos_fld
1B    0.990838
2B    0.981784
3B    0.962512
C     0.990847
OF    0.983018
P     0.947876
SS    0.972208
Name: Fld%, dtype: float64

### Superior Rating

In [488]:
players["superior_rating"] = ""
players.loc[(players["Primary_Pos_fld"] == "P") & (players["Fld%"] >= 0.980), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "C") & (players["Fld%"] >= 0.993), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "1B") & (players["Fld%"] >= 0.995), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "2B") & (players["Fld%"] >= 0.984), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "3B") & (players["Fld%"] >= 0.971), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "SS") & (players["Fld%"] >= 0.973), "superior_rating"] = "S"
players.loc[(players["Primary_Pos_fld"] == "OF") & (players["Fld%"] >= 0.990), "superior_rating"] = "S"

In [489]:
players["superior_rating"].value_counts()

S    883
     748
Name: superior_rating, dtype: int64

In [490]:
players.head(10)

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,Rdrs/yr_cat,Rgood_cat,Rair,Rrange,Rthrow,RszC,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating
0,Fernando Abad,31,BOS,AL,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1,L,AL,2.0,1.0,0.667,3.30,48.0,0.0,15.0,0.0,0.0,1.0,43.2,40.0,18.0,16.0,4.0,14.0,1.0,37.0,1.0,1.0,0.0,182.0,139.0,3.68,1.237,8.2,0.8,2.9,7.6,2.64,L,BOS,AL,48.0,0.0,0.0,43.2,6.0,0.0,6.0,0.0,1.0,1.000,NaN,NaN,1.0,5.0,0.0,1.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BOS,8,48,0,4,48,48,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.240,M,43.666667,1.0,3.0,13.0,7.0,10.0,24,0,0.0,NaN,NaN,0.100000,+,11.0,51.0,8.0,55.0,+M1,(13-24),+M1 (13-24),S
1,José Abreu,30,CHW,AL,156.0,675.0,621.0,95.0,189.0,43.0,6.0,33.0,102.0,3.0,0.0,35.0,119.0,0.304,0.354,0.552,0.906,141.0,343.0,21.0,15.0,0.0,4.0,6.0,*3D,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHW,AL,139.0,138.0,130.0,1197.0,1221.0,1135.0,78.0,8.0,130.0,0.993,3.0,3.0,2.0,2.0,-1.0,9.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,CHW,4,156,156,156,139,0,0,139,0,0,0,0,0,0,0,18,0,0,1B,0.653846,#,B+,6,16.0,1,7.0,(21),0.019108,NaN,2,12,6,8.0,22,1,9.0,/23,10,53,#B+16(21) (12-22/23),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,Cristhian Adames,25,COL,NL,12.0,14.0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.000,0.071,0.000,0.071,-80.0,0.0,0.0,0.0,0.0,0.0,0.0,H/346,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COL,NL,3.0,1.0,0.0,9.0,4.0,3.0,1.0,0.0,0.0,1.000,-1.0,-93.0,-1.0,-133.0,0.0,4.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,COL,4,12,1,12,2,0,0,1,1,0,1,0,0,0,0,0,10,0,SS,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000000,NaN,3,13,15,18.0,36,0,0.0,NaN,0,66,G (13-36),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
3,Austin Adams,26,WSN,NL,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,/1,R,NL,0.0,0.0,NaN,3.60,6.0,0.0,3.0,0.0,0.0,0.0,5.0,4.0,4.0,2.0,0.0,8.0,0.0,10.0,1.0,0.0,1.0,29.0,133.0,4.56,2.400,7.2,0.0,14.4,18.0,1.25,R,WSN,NL,6.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.000,NaN,NaN,0.0,0.0,0.0,1.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,WSN,1st,6,0,6,6,6,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,G,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,0,0.0,n,0,0.0,NaN,0,66,G (n-n),0.200,L,5.000000,1.0,10.0,24.0,12.0,22.0,44,1,23.0,/45,NaN,0.000000,-,16.0,42.0,5.0,62.0,-L1,(24-44/45),-L1 (24-44/45),S
4,Lane Adams,27,ATL,NL,85.0,122.0,109.0,19.0,30.0,4.0,1.0,5.0,20.0,10.0,0.0,10.0,37.0,0.275,0.339,0.468,0.807,109.0,51.0,3.0,1.0,1.0,1.0,0.0,H78/9,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,NL,41.0,13.0,9.0,172.0,51.0,48.0,1.0,2.0,0.0,0.961,1.0,5.0,NaN,NaN,NaN,2.56,...

### Arm Rating

In [491]:
players["G"].value_counts()

10.0     38
27.0     35
5.0      33
1.0      33
4.0      33
         ..
140.0     1
104.0     1
80.0      1
129.0     1
159.0     1
Name: G, Length: 158, dtype: int64

In [492]:
players["a_gp"] = players["A"] / players["G_app"]
players["a_gp"].mean()

0.45898909057743714

In [493]:
players["arm_rating"] = 8
players.loc[(players["Primary_Pos_fld"] == "P") & (players["a_gp"] >= 0.7), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "C"), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "1B") & (players["a_gp"] >= 0.7), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "2B") & (players["a_gp"] >= 2.8), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "3B") & (players["a_gp"] >= 2.0), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "SS") & (players["a_gp"] >= 2.8), "arm_rating"] = 9
players.loc[(players["Primary_Pos_fld"] == "OF") & (players["a_gp"] >= 0.08), "arm_rating"] = 9

In [494]:
players["arm_rating"].value_counts()

8    1339
9     292
Name: arm_rating, dtype: int64

### Range Rating

In [495]:
players["po_gp"] = players["PO"] / players["G_app"]
players["po_gp"].mean()

1.1035792567759144

In [496]:
players.groupby("Primary_Pos_fld")["po_gp"].mean()

Primary_Pos_fld
1B    4.789588
2B    1.144500
3B    0.842025
C     5.689737
OF    1.394689
P     0.134144
SS    1.063347
Name: po_gp, dtype: float64

In [497]:
players["range_rating"] = 4
players.loc[(players["Primary_Pos_fld"] == "P") & (players["po_gp"] >= 0.3), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "C"), "range_rating"] = 4
players.loc[(players["Primary_Pos_fld"] == "1B") & (players["po_gp"] >= 8.3), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "2B") & (players["po_gp"] >= 2.1), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "3B") & (players["po_gp"] >= 0.8), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "SS") & (players["po_gp"] >= 1.6), "range_rating"] = 5
players.loc[(players["Primary_Pos_fld"] == "OF") & (players["po_gp"] >= 2.1), "range_rating"] = 5

In [498]:
players["range_rating"].value_counts()

4    1445
5     186
Name: range_rating, dtype: int64

### Catcher Caught Stealing Rate

In [499]:
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'Rdrs', 'Rdrs/yr', 'Rgood', 'RF/9',
       'RF/G', 'Pos_Summary_fld', 'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat',
       'CG_cat', 'Inn_cat', 'Ch_cat', 'PO_cat', 'A_cat', 'E_cat', 'DP_cat',
       'Fld%_cat', 'Rtot_cat', 'Rtot/yr_cat', 'Rctch', 'Rdrs_cat',
       'Rdrs/yr_cat', 'Rgood_cat', 'Rair', 'Rrange', 'Rthrow', 'RszC', 'RsbC',
       'RerC', 'RF/9_cat

In [500]:
players["CS%"].value_counts()

0%      16
20%      7
25%      6
27%      6
23%      5
33%      5
24%      5
17%      5
37%      5
40%      4
8%       4
14%      4
30%      3
42%      3
29%      3
38%      3
31%      3
26%      2
21%      2
39%      2
35%      2
10%      2
50%      2
32%      2
13%      2
44%      2
36%      2
28%      1
45%      1
19%      1
18%      1
49%      1
43%      1
6%       1
100%     1
11%      1
Name: CS%, dtype: int64

In [501]:
players["cs_rate"] = players["CS_cat"] / (players["SB_cat"] + players["CS_cat"])
players["cs_rate"].mean()

0.24205786589049944

In [502]:
cs_break_points = [
    0.21,
    0.31,
    0.41,
    0.51
]

rating = [
    "",
    "-1",
    "-2",
    "-3",
    "-4"
]

def cs_rating(cs_rate, breakpoints=cs_break_points, ratings=rating):
    i = bisect(breakpoints, cs_rate)
    return ratings[i]

In [503]:
players["cs_num"] = [cs_rating(rate) for rate in players["cs_rate"]]
players["cs_num"].value_counts()

-4    1516
        44
-1      36
-2      25
-3      10
Name: cs_num, dtype: int64

In [504]:
players["G_cat"].isnull().sum()

1506

In [505]:
players.loc[(players["G_cat"].isnull()), "cs_num"] = ""
players.loc[(players["SB_cat"] == 0), "cs_num"] = ""
players["cs_num"].value_counts()

      1560
-1      36
-2      25
-3      10
Name: cs_num, dtype: int64

In [506]:
players[players["cs_num"] == "-4"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,RsbC,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating,a_gp,arm_rating,po_gp,range_rating,cs_rate,cs_num


### Fielder Rating

In [507]:
players["fielder_rating"] = (players["superior_rating"] + 
                             players["arm_rating"].astype(str) +
                             players["range_rating"].astype(str) + 
                             " " + 
                             players["cs_num"]
)
players["fielder_rating"].value_counts()

S84       679
84        527
94         98
85         73
S94        70
S85        59
S95        28
95         26
S94 -1     24
S94 -2     17
94 -1      12
94 -2       7
S94 -3      6
94 -3       4
84 -2       1
Name: fielder_rating, dtype: int64

In [508]:
players[players["fielder_rating"] == "95 -2"]

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,Rdrs,Rdrs/yr,Rgood,RF/9,...,RerC,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating,a_gp,arm_rating,po_gp,range_rating,cs_rate,cs_num,fielder_rating


In [509]:
players.to_csv("../data/player stats - " + year + " - with batter pitcher and fielder ratings.csv", index=False)

# Save teams to separate Excel tabs

In [8]:
players = pd.read_csv("../data/player stats - " + year + " - with batter pitcher and fielder ratings.csv")

In [9]:
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,...,bb_k_hbp,pitcher_rating,superior_rating,a_gp,arm_rating,po_gp,range_rating,cs_rate,cs_num,fielder_rating
0,Jim Acker,27,TOT,MLB,21.0,28.0,28.0,1.0,3.0,1.0,...,(13-21),W4 (13-21) [WP],S,0.636364,8,0.363636,5,NaN,NaN,S85
1,Jim Acker,27,ATL,NL,21.0,28.0,28.0,1.0,3.0,1.0,...,(12-15),W5 (12-15),S,0.636364,8,0.363636,5,NaN,NaN,S85
2,Jim Adduci,26,MIL,AL,3.0,13.0,11.0,2.0,1.0,1.0,...,NaN,NaN,S,1.000000,9,8.333333,5,NaN,NaN,S95
3,Luis Aguayo,27,PHI,NL,62.0,146.0,133.0,17.0,28.0,6.0,...,NaN,NaN,NaN,1.451613,8,0.919355,4,NaN,NaN,84
4,Rick Aguilera,24,NYM,NL,32.0,57.0,51.0,4.0,8.0,0.0,...,(12-22),+W5 (12-22) [WP],S,0.812500,9,0.406250,5,NaN,NaN,S95


In [10]:
pd.set_option('display.max_seq_items', 175)
players.columns

Index(['Name', 'Age', 'Tm', 'Lg_bat', 'G_bat', 'PA', 'AB', 'R_bat', 'H_bat',
       '2B', '3B', 'HR_bat', 'RBI', 'SB', 'CS', 'BB_bat', 'SO_bat', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP_bat', 'SH', 'SF',
       'IBB_bat', 'Pos_Summary', 'Bats', 'Lg_pit', 'W', 'L', 'W-L%', 'ERA',
       'G_pit', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H_pit', 'R_pit', 'ER',
       'HR_pit', 'BB_pit', 'IBB_pit', 'SO_pit', 'HBP_pit', 'BK', 'WP', 'BF',
       'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'Throws',
       'Tm_fld', 'Lg', 'G', 'GS_fld', 'CG_fld', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'RF/9', 'RF/G', 'Pos_Summary_fld',
       'Tm_cat', 'Lg_cat', 'G_cat', 'GS_cat', 'CG_cat', 'Inn_cat', 'Ch_cat',
       'PO_cat', 'A_cat', 'E_cat', 'DP_cat', 'Fld%_cat', 'Rtot_cat',
       'Rtot/yr_cat', 'Rctch', 'RF/9_cat', 'RF/G_cat', 'PB', 'WP_cat',
       'SB_cat', 'CS_cat', 'CS%', 'Rk', 'Tm_app', 'Yrs', 'G_app', 'GS_app',
       'Batting', 'Defense'

In [11]:
pd.set_option('display.max_columns', 175)
players.head()

,Name,Age,Tm,Lg_bat,G_bat,PA,AB,R_bat,H_bat,2B,3B,HR_bat,RBI,SB,CS,BB_bat,SO_bat,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP_bat,SH,SF,IBB_bat,Pos_Summary,Bats,Lg_pit,W,L,W-L%,ERA,G_pit,GS,GF,CG,SHO,SV,IP,H_pit,R_pit,ER,HR_pit,BB_pit,IBB_pit,SO_pit,HBP_pit,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Throws,Tm_fld,Lg,G,GS_fld,CG_fld,Inn,Ch,PO,A,E,DP,Fld%,Rtot,Rtot/yr,RF/9,RF/G,Pos_Summary_fld,Tm_cat,Lg_cat,G_cat,GS_cat,CG_cat,Inn_cat,Ch_cat,PO_cat,A_cat,E_cat,DP_cat,Fld%_cat,Rtot_cat,Rtot/yr_cat,Rctch,RF/9_cat,RF/G_cat,PB,WP_cat,SB_cat,CS_cat,CS%,Rk,Tm_app,Yrs,G_app,GS_app,Batting,Defense,P,C,1B,2B_app,3B_app,SS,LF,CF,RF,OF,DH,PH,PR,Primary_Pos_fld,rbi_per_g,clutch,bat_letter,hr_rate,hr_num_bat,triple_rate,triple_val,triple_num,speed_score,speed_rating,walk_rate,bb_num,k_rate,k_val,k_num,hbp_rate,hbp_val,hbp_num,hit_rate,PH_num_bat,batter_rating,BAA,pit_letter,IP_real,IE,bb_rate,bb_num_pit,k_rate_pit,k_val_pit,k_num_pit,hbp_rate_pit,hbp_val_pit,hbp_num_pit,WP_num,hr_rate_pit,gopher_ball,control_rate,PCN,hit_rate_pit,PPH,goph_lett_inn,bb_k_hbp,pitcher_rating,superior_rating,a_gp,arm_rating,po_gp,range_rating,cs_rate,cs_num,fielder_rating
0,Jim Acker,27,TOT,MLB,21.0,28.0,28.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.107,0.107,0.143,0.250,-33.0,4.0,0.0,0.0,0.0,0.0,0.0,1,R,MLB,5.0,12.0,0.294,4.01,44.0,19.0,9.0,0.0,0.0,0.0,155.0,163.0,81.0,69.0,13.0,48.0,6.0,69.0,3.0,1.0,5.0,661.0,102.0,3.96,1.361,9.5,0.8,2.8,4.0,1.44,R,TOT,ZZ,44.0,19.0,0.0,155.0,44.0,16.0,28.0,0.0,4.0,1.000,NaN,NaN,2.55,1.00,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2TM,4,44,19,21,44,44,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,E,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,27,27.0,53,0,0.0,NaN,4,63,E (n-53),0.267,W,155.000000,4.0,3.0,13.0,4.0,7.0,21,0,0.0,NaN,[WP],0.079755,NaN,12.0,46.0,9.0,54.0,W4,(13-21),W4 (13-21) [WP],S,0.636364,8,0.363636,5,NaN,NaN,S85
1,Jim Acker,27,ATL,NL,21.0,28.0,28.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.107,0.107,0.143,0.250,-33.0,4.0,0.0,0.0,0.0,0.0,0.0,1,R,NL,3.0,8.0,0.273,3.79,21.0,14.0,3.0,0.0,0.0,0.0,95.0,100.0,47.0,40.0,7.0,26.0,3.0,37.0,1.0,0.0,2.0,402.0,105.0,3.80,1.326,9.5,0.7,2.5,3.5,1.42,R,TOT,ZZ,44.0,19.0,0.0,155.0,44.0,16.0,28.0,0.0,4.0,1.000,NaN,NaN,2.55,1.00,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2TM,4,44,19,21,44,44,0,0,0,0,0,0,0,0,0,0,0,0,P,0.000000,NaN,E,0,NaN,0,0.0,NaN,0.000000,NaN,0,n,27,27.0,53,0,0.0,NaN,4,63,E (n-53),0.267,W,95.000000,5.0,2.0,12.0,3.0,5.0,15,0,0.0,NaN,NaN,0.070000,NaN,11.0,51.0,9.0,54.0,W5,(12-15),W5 (12-15),S,0.636364,8,0.363636,5,NaN,NaN,S85
2,Jim Adduci,26,MIL,AL,3.0,13.0,11.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.091,0.167,0.182,0.348,-6.0,2.0,0.0,0.0,1.0,0.0,0.0,/3,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MIL,AL,3.0,3.0,3.0,24.2,28.0,25.0,3.0,0.0,1.0,1.000,0.0,-15.0,10.22,9.33,1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,MIL,2,3,3,3,3,0,0,3,0,0,0,0,0,0,0,0,0,0,1B,0.000000,NaN,E,0,NaN,0,0.0,NaN,0.000000,NaN,3,13,6,9.0,23,0,0.0,NaN,3,64,E (13-23),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,1.000000,9,8.333333,5,NaN,NaN,S95
3,Luis Aguayo,27,PHI,NL,62.0,146.0,133.0,17.0,28.0,6.0,1.0,4.0,13.0,1.0,1.0,8.0,26.0,0.211,0.267,0.361,0.628,70.0,48.0,3.0,3.0,0.0,2.0,0.0,46H/5,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHI,NL,52.0,30.0,22.0,300.2,152.0,57.0,90.0,5.0,19.0,0.967,-4.0,-17.0,4.40,2.83,2B-SS-3B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,PHI,7,62,30,62,49,0,0,0,31,1,20,0,0,0,0,0,16,3,2B,0.209677,NaN,C,5,15.0,1,6.0,(16),0.035714,NaN,2,12,6,8.0,22,1,9.0,/23,7,56,C15(16) (12-22/23),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.451613,8,0.919355,4,NaN,NaN,84
4,Ri

In [12]:
# fix games played column
players.loc[(players["Primary_Pos_fld"] == "P"), "Games_Played"] = players["G_pit"]
players.loc[(players["Primary_Pos_fld"] != "P"), "Games_Played"] = players["G_bat"]
players["Games_Played"].value_counts()

5.0      24
34.0     23
6.0      23
10.0     23
16.0     21
         ..
129.0     1
121.0     1
47.0      1
131.0     1
133.0     1
Name: Games_Played, Length: 162, dtype: int64

In [18]:
players_short = players.loc[:, ["Name", "Age", "Tm", "Games_Played", "GS", "GF", "Pos_Summary_fld", 
                                "fielder_rating", "batter_rating", "PH_num_bat", "pitcher_rating", "PCN", "PPH", 
                                "Bats", "Throws", "Primary_Pos_fld", "P", "C", "1B", "2B_app", "3B_app", "SS", 
                                "LF", "CF", "RF", "OF", "DH", "PH", "PR"]]
players_short.rename(columns={
    "Games_Played": "G",
    "Pos_Summary_fld": "Positions",
    "fielder_rating": "DEF",
    "PH_num_bat": "BPH",
    "Bats": "B",
    "Throws": "T",
    "Primary_Pos_fld": "Primary",
    "2B_app": "2B",
    "3B_app": "3B"
}, inplace=True)
players_short.head()

,Name,Age,Tm,G,GS,GF,Positions,DEF,batter_rating,BPH,pitcher_rating,PCN,PPH,B,T,Primary,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR
0,Jim Acker,27,TOT,44.0,19.0,9.0,P,S85,E (n-53),63,W4 (13-21) [WP],46.0,54.0,R,R,P,44,0,0,0,0,0,0,0,0,0,0,0,0
1,Jim Acker,27,ATL,21.0,14.0,3.0,P,S85,E (n-53),63,W5 (12-15),51.0,54.0,R,R,P,44,0,0,0,0,0,0,0,0,0,0,0,0
2,Jim Adduci,26,MIL,3.0,NaN,NaN,1B,S95,E (13-23),64,NaN,NaN,NaN,L,NaN,1B,0,0,3,0,0,0,0,0,0,0,0,0,0
3,Luis Aguayo,27,PHI,62.0,NaN,NaN,2B-SS-3B,84,C15(16) (12-22/23),56,NaN,NaN,NaN,R,NaN,2B,0,0,0,31,1,20,0,0,0,0,0,16,3
4,Rick Aguilera,24,NYM,28.0,20.0,2.0,P,S95,D23 (12-24),62,+W5 (12-22) [WP],51.0,54.0,R,R,P,28,0,0,0,0,0,0,0,0,0,0,4,0


In [19]:
players_short["Primary"].replace(np.nan, "DH_PH_PR", inplace=True)

In [20]:
pos_cat_dtype = pd.api.types.CategoricalDtype(categories=["C", "1B", "2B", "3B", "SS", "OF", "DH_PH_PR", "P", ""],
                                              ordered=True) 
players_short["Primary"] = players_short['Primary'].astype(pos_cat_dtype)
players_short["Primary"].value_counts()

P           487
OF          232
C            96
1B           75
SS           62
3B           62
2B           62
DH_PH_PR      7
              0
Name: Primary, dtype: int64

In [21]:
players_short.fillna("", inplace=True)
players_short.head()

,Name,Age,Tm,G,GS,GF,Positions,DEF,batter_rating,BPH,pitcher_rating,PCN,PPH,B,T,Primary,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR
0,Jim Acker,27,TOT,44.0,19,9,P,S85,E (n-53),63,W4 (13-21) [WP],46,54,R,R,P,44,0,0,0,0,0,0,0,0,0,0,0,0
1,Jim Acker,27,ATL,21.0,14,3,P,S85,E (n-53),63,W5 (12-15),51,54,R,R,P,44,0,0,0,0,0,0,0,0,0,0,0,0
2,Jim Adduci,26,MIL,3.0,,,1B,S95,E (13-23),64,,,,L,,1B,0,0,3,0,0,0,0,0,0,0,0,0,0
3,Luis Aguayo,27,PHI,62.0,,,2B-SS-3B,84,C15(16) (12-22/23),56,,,,R,,2B,0,0,0,31,1,20,0,0,0,0,0,16,3
4,Rick Aguilera,24,NYM,28.0,20,2,P,S95,D23 (12-24),62,+W5 (12-22) [WP],51,54,R,R,P,28,0,0,0,0,0,0,0,0,0,0,4,0


In [32]:
players_short["Name"] = players_short["Name"].str.replace("\xa0", " ")

In [33]:
players_short[players_short["Name"] == "Steve Carlton"]

,Name,Age,Tm,G,GS,GF,Positions,DEF,batter_rating,BPH,pitcher_rating,PCN,PPH,B,T,Primary,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR
114,Steve Carlton,41,TOT,32.0,32,0,P,S94,C14 (11-16),56,+W6 (14-23) [WP],45,54,L,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0
115,Steve Carlton,41,TOT,22.0,22,0,P,S94,C14 (11-16),56,+X5 (14-23) [WP],44,54,L,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0
116,Steve Carlton,41,TOT,32.0,32,0,P,S94,C14 (11-16),56,+W6 (14-23) [WP],45,54,L,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0
117,Steve Carlton,41,TOT,22.0,22,0,P,S94,C14 (11-16),56,+X5 (14-23) [WP],44,54,L,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0
118,Steve Carlton,41,PHI,16.0,16,0,P,S94,C (11-15),56,+X5 (14-24),45,54,L,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0
119,Steve Carlton,41,SFG,6.0,6,0,P,S94,D+36 (n-24),56,+X5 (14-23),44,54,L,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0
901,Steve Carlton,41,CHW,10.0,10,0,P,S94,G (n-n),66,+M6 (13-23),46,55,,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
players_short.drop_duplicates(["Name", "Tm"], keep='first', inplace=True)

In [36]:
players_short[players_short["Name"] == "Steve Carlton"]

,Name,Age,Tm,G,GS,GF,Positions,DEF,batter_rating,BPH,pitcher_rating,PCN,PPH,B,T,Primary,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR
114,Steve Carlton,41,TOT,32.0,32,0,P,S94,C14 (11-16),56,+W6 (14-23) [WP],45,54,L,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0
118,Steve Carlton,41,PHI,16.0,16,0,P,S94,C (11-15),56,+X5 (14-24),45,54,L,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0
119,Steve Carlton,41,SFG,6.0,6,0,P,S94,D+36 (n-24),56,+X5 (14-23),44,54,L,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0
901,Steve Carlton,41,CHW,10.0,10,0,P,S94,G (n-n),66,+M6 (13-23),46,55,,L,P,32,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
players_short = players_short.sort_values(["Tm", "Primary", "G", "GS", "GF"], 
                                          ascending = (True, True, False, False, False))
players_short.head(50)

,Name,Age,Tm,G,GS,GF,Positions,DEF,batter_rating,BPH,pitcher_rating,PCN,PPH,B,T,Primary,P,C,1B,2B,3B,SS,LF,CF,RF,OF,DH,PH,PR
796,Ozzie Virgil,29,ATL,114.0,,,C,94 -1,C+21 (15-25),56,,,,R,,C,0,111,0,0,0,0,0,0,0,0,0,5,0
48,Bruce Benedict,30,ATL,64.0,,,C,S94 -2,C+(11) (13-15),56,,,,R,,C,0,57,0,0,0,0,0,0,0,0,0,7,0
338,Bob Horner,28,ATL,141.0,,,1B,S95,#B21 (13-21),54,,,,R,,1B,0,0,139,0,0,0,0,0,0,0,0,4,0
128,Chris Chambliss,37,ATL,97.0,,,1B,84,A12 (14-24),53,,,,L,,1B,0,0,20,0,0,0,0,0,0,0,0,80,0
705,Ted Simmons,36,ATL,76.0,,,1B-C-3B,84,B14 (13-21),55,,,,S,,1B,0,10,14,0,9,0,0,0,0,0,0,54,0
346,Glenn Hubbard,28,ATL,143.0,,,2B,94,C+12 (15-24),56,,,,R,,2B,0,0,0,142,0,0,0,0,0,0,0,1,0
663,Paul Runge,28,ATL,7.0,,,2B,S84,C+ (21-43),56,,,,R,,2B,0,0,0,6,0,0,0,0,0,0,0,0,3
556,Ken Oberkfell,30,ATL,151.0,,,3B-2B,S94,B11(12) (15-21),55,,,,L,,3B,0,0,0,41,130,0,0,0,0,0,0,6,0
404,Brad Komminsk,25,ATL,5.0,,,3B-OF,S84,AAA (n-21),45,,,,R,,3B,0,0,0,0,2,0,0,0,2,2,0,1,1
620,Rafael Ramirez,28,ATL,134.0,,,SS-3B-OF,84,C+12** (11-15),55,,,,R,,SS,0,0,0,0,58,87,2,0,1,3,0,6,3


In [38]:
my_dict = {index: group_teams for index, group_teams in players_short.groupby('Tm')}
my_dict

{'ATL':                     Name  Age   Tm      G  GS  GF Positions     DEF  \
 796         Ozzie Virgil   29  ATL  114.0                 C   94 -1   
 48        Bruce Benedict   30  ATL   64.0                 C  S94 -2   
 338           Bob Horner   28  ATL  141.0                1B    S95    
 128      Chris Chambliss   37  ATL   97.0                1B     84    
 705          Ted Simmons   36  ATL   76.0           1B-C-3B     84    
 346        Glenn Hubbard   28  ATL  143.0                2B     94    
 663           Paul Runge   28  ATL    7.0                2B    S84    
 556        Ken Oberkfell   30  ATL  151.0             3B-2B    S94    
 404        Brad Komminsk   25  ATL    5.0             3B-OF    S84    
 620       Rafael Ramirez   28  ATL  134.0          SS-3B-OF     84    
 762        Andres Thomas   22  ATL  102.0                SS     94    
 536          Dale Murphy   30  ATL  160.0                OF     84    
 522          Omar Moreno   33  ATL  118.0               

In [39]:
# https://stackoverflow.com/questions/21981820/creating-multiple-excel-worksheets-using-data-in-a-pandas-dataframe/21984437
writer = pd.ExcelWriter('../data/' + year + ' rosters ' + '.xlsx', engine='xlsxwriter')

def create_excel(dictionary):
    count = 0
    for i, j in dictionary.items():
        dictionary[i].to_excel(writer, sheet_name=i)
        count += 1
    
    writer.save()
    return count

In [40]:
create_excel(my_dict)

27